In [ ]:
#Start Here for Making Beam-Date Graphs
import os
os.chdir('analysis/multi_freq_from_archive')
#format beamsize number
def format_number(number):
        # Case 1: If the number is less than 1 (e.g., 0.004567)
        if number < 0:
                # Format the positive part of the number and prepend the negative sign
                return "-" + format_number(abs(number))
        if number < 1:
                # Convert the number to a string with high precision
                num_str = f"{number:.16g}"

                # Identify the leading zeros and decimal point
                leading_part = []
                for char in num_str:
                        if char == '0' or char == '.':
                                leading_part.append(char)
                        else:
                                break

                # Remove leading zeros and the decimal point for significant digits
                significant_digits = "".join(char for char in num_str if char.isdigit() and char != "0")

                # Extract the first two non-zero digits
                if len(significant_digits) >= 2:
                        first_two = significant_digits[:2]
                elif len(significant_digits) == 1:
                        first_two = significant_digits + "0"  # Pad with zero if only one significant digit exists
                else:
                        first_two = "00"  # Handle edge case like 0

                # Combine the leading zeros and the first two non-zero digits
                return "".join(leading_part) + first_two

        # Case 2: If the number is greater than or equal to 1
        else:
                # Format to two decimal places
                return f"{number:.2f}"
        

In [ ]:
import numpy as np

def linear_to_log_uncertainty(x, delta_x):
    if x == 0:
        return np.nan   # or 0, depending on what makes sense
    return delta_x / (x * np.log(10))

def log_to_linear_uncertainty(logx, delta_logx):
    """Convert log10 uncertainty Δlogx back to linear uncertainty Δx.
    
    Parameters
    ----------
    logx : float or array
        Value in log10-space.
    delta_logx : float or array
        Uncertainty in log10(x).
    """
    x = 10**logx
    return x * np.log(10) * delta_logx

In [ ]:
#Create Spectral Plots
import matplotlib.pyplot as pylab
import matplotlib.colors
import numpy as np
import shutil
import os
import matplotlib.cm as cmx
import math
from matplotlib.ticker import MultipleLocator
import copy

def round_down_to_half_integer(num):
    return math.floor(num * 2) / 2

def round_up_to_half_integer(num):
    return math.ceil(num * 2) / 2

def round_down_to_tenth_integer(num):
    return math.floor(num * 10) / 10

def round_up_to_tenth_integer(num):
    return math.ceil(num * 10) / 10

msize=10

path = f'SpectraGraphs'
if os.path.exists(path):
    shutil.rmtree(path)
os.mkdir(path)

source_text='fitsumfiles/final/fitsummary_final_withextras.txt'
freqs=[]
fluxs=[]
beams=[]

with open(source_text, 'r') as infile:
    jfin=-1
    for line in infile:
        if len(line)<2:
            continue
        jfin=jfin+1 

grouped_data = {}

with open(source_text, 'r') as infile:
    j=-1
    for line in infile:
        if line=='\n':
            continue
        
        line=eval(line)
        source=line[0]
        breaker=1

        j=j+1
        
        freq=np.log10(float(line[2]))
        flux=np.log10(float(str(line[3]).split('*')[0]))

        nobeam=0
        if '-' not in line[5]:
            beam=np.log10(float(line[5]))
        else:
            beam='NA'
            nobeam=1


        if source not in grouped_data:
            grouped_data[source] = {
                'sfreqs': [], 'sfluxs': [], 'sbeams': [], 'sdates': [], 
                'sfreqs2': [], 'sfluxs2': [], 'sbeams2': [], 'sdates2': [],
                'sfiles': [], 'sfiles2': []
            }

            sbeamsizes=[]
            sfluxs=[]
            sfreqs=[]
            sdates=[]
            sfiles=[]

            sbeamsizes2=[]
            sfluxs2=[]
            sfreqs2=[]
            sdates2=[]
            sfiles2=[]

        if j==0:
            minfreq=freq
            maxfreq=freq
            minflux=flux
            maxflux=flux
            if nobeam==0:
                minbeam=beam
                maxbeam=beam
        if j!=0:
            if freq>maxfreq:
                maxfreq=freq
            if freq<minfreq:
                minfreq=freq
            if flux>maxflux:
                maxflux=flux
            if flux<minflux:
                minflux=flux
            if nobeam==0:
                if beam>maxbeam:
                    maxbeam=beam
                if beam<minbeam:
                    minbeam=beam

minfreqp=round_down_to_half_integer(minfreq)
maxfreqp=round_up_to_half_integer(maxfreq)

minfluxp=round_down_to_half_integer(minflux)
maxfluxp=round_up_to_half_integer(maxflux)

minbeamp=round_down_to_half_integer(float(minbeam))
maxbeamp=round_up_to_half_integer(float(maxbeam))

interval = maxfreqp - minfreqp
ten_percent = interval * 0.05
minfreq = minfreqp - ten_percent
maxfreq = maxfreqp + ten_percent

interval = maxfluxp - minfluxp
ten_percent = interval * 0.05
minflux = minfluxp - ten_percent
maxflux = maxfluxp + ten_percent

interval = maxbeamp - minbeamp
ten_percent = interval * 0.05
minbeam = minbeamp - ten_percent
maxbeam = maxbeamp + ten_percent


x_tick_positions=np.arange(minfreqp, maxfreqp + 0.5 , 0.5)  
x_tick_labels = [f"{pos:.2f}" for pos in x_tick_positions] 

y_tick_positions = np.arange(minfluxp, maxfluxp + 0.5, 0.5)  
y_tick_labels = [f"{pos:.2f}" for pos in y_tick_positions]  

stick_positions = np.arange(minbeamp, maxbeamp + 1, 1)  
stick_labels = [f"{pos:.2f}" for pos in stick_positions] 

source_text='fitsumfiles/final/fitsummary_final_withextras.txt'

sourcedone=[]
sources=[]

oldsources=[]
oldsource='NA'

with open(source_text, 'r') as infile:
    j=-1
    j2=-1
    for line in infile:
        if len(line)<2:
            continue
        j2=j2+1 
        line=eval(line)

        source=line[0]
        breaker=1
        j=j+1

        if j==0:
            sbeamsizes=[]
            sdates=[]
            sfluxs=[]
            sfreqs=[]
            
            sfiles=[]
            sfiles2=[]

            sbeamsizes2=[]
            sdates2=[]
            sfluxs2=[]
            sfreqs2=[]

        sources.append(source)
        sources=list(set(sources))

        if len(sources)-len(oldsources)!=0 and j!=0:
            source=oldsource
            sourcedone.append(source)

            fig, ax = pylab.subplots(dpi=300)
            cm2 = pylab.get_cmap('jet')   
            sNorm = matplotlib.colors.Normalize(vmin=minbeam, vmax=maxbeam)
            scalarMap2 = cmx.ScalarMappable(norm=sNorm, cmap=cm2)

            def assign_colors(beamsizes):
                colors = []
                for size in beamsizes:
                    if isinstance(size, list):
                        size = size[0]
                    colors.append(scalarMap2.to_rgba(size))  # Use colormap for valid sizes
                return colors
            
            colors2 = assign_colors(sbeamsizes2)
            colors = assign_colors(sbeamsizes)

            if len(sfreqs2)!=0:
                pylab.scatter(sfreqs2,sfluxs2, c=colors2,marker='v',s=msize) 

            if len(sfreqs)!=0:
                flux_vals = [float(f[0]) for f in sfluxs]
                unc_vals  = [float(f[1]) for f in sfluxs]

                for f, fl, err, c in zip(sfreqs, flux_vals, unc_vals, colors):
                    pylab.errorbar(f, fl,
                                    yerr=err,
                                    fmt='o', color=c,
                                    markersize=2, capsize=0)


            colorbar1 = fig.colorbar(
                    cmx.ScalarMappable(norm=matplotlib.colors.Normalize(vmin=minbeamp, vmax=maxbeamp), cmap=cm2),
                    label='Beamsize ["]',
                    ax=ax,
                    ticks=stick_positions
                    )
            
            colorbar1.set_label(
                r'log$_{10}$ Beamsize ["]'
            )

            minor_tick_locations = np.arange(minbeamp, maxbeamp, 0.1)
            major_ticks = colorbar1.get_ticks()
            filtered_minor_ticks = [tick for tick in minor_tick_locations if tick not in major_ticks]
            colorbar1.ax.yaxis.set_ticks(filtered_minor_ticks, minor=True)
            
            ax = pylab.gca()
            ax.set_xlim([minfreq, maxfreq])
            ax.set_xticks(x_tick_positions)
            ax.set_xticklabels(x_tick_labels)

            ax.set_ylim([minflux, maxflux]) 
            ax.set_yticks(y_tick_positions)
            ax.set_yticklabels(y_tick_labels)

            minor_locator = MultipleLocator(0.1)
            ax.xaxis.set_minor_locator(minor_locator)
            ax.yaxis.set_minor_locator(minor_locator)
                                
            pylab.yticks(y_tick_positions, y_tick_labels)
            pylab.xlabel(r'log$_{10}$ Frequency [GHz]')
            pylab.title(f'{source}',fontsize=20)
            pylab.ylabel(r'log$_{10}$ Flux [mJy][Beam]$^{-1}$')
            nicename=f'{source} Spectra.pdf'
            if not os.path.exists(f'SpectraGraphs'):
                os.makedirs(f'SpectraGraphs')
            if os.path.exists(f'SpectraGraphs/{nicename}'):
                os.remove(f'SpectraGraphs/{nicename}')
            pylab.savefig(f'SpectraGraphs/{nicename}', dpi=300)
            pylab.show()
            pylab.clf() 

            if len(sfreqs)!=0:
                grouped_data[source]['sfreqs']=sfreqs
                grouped_data[source]['sfluxs']=sfluxs
                grouped_data[source]['sbeams']=sbeamsizes
                grouped_data[source]['sdates']=sdates
                grouped_data[source]['sfiles']=sfiles

                grouped_data[source]['sfreqs2']=sfreqs2
                grouped_data[source]['sfluxs2']=sfluxs2
                grouped_data[source]['sbeams2']=sbeamsizes2
                grouped_data[source]['sdates2']=sdates2
                grouped_data[source]['sfiles2']=sfiles2


            sbeamsizes=[]
            sfluxs=[]
            sfreqs=[]
            sdates=[]

            sfiles=[]
            sfiles2=[]

            sbeamsizes2=[]
            sfluxs2=[]
            sfreqs2=[]
            sdates2=[]

        oldsource=source

        flux_upperb=0
        if len(str(line[3]).split('*'))>1:
            flux_upperb=1

        freq=np.log10(float(line[2]))
        flux=np.log10(float(str(line[3]).split('*')[0]))
        fluxunc=line[6]
        if fluxunc!='NA':
            fluxunc=float(line[3])/float(line[6])
            fluxunc=linear_to_log_uncertainty(float(line[3]),float(fluxunc))
        odate=line[4]
        filename=line[7]
        if odate!='NA':
            odate=odate.split('/')
            date=float(odate[0]) + (float(odate[1])-1) / 12 + (float(odate[2])-1) / 365
        else:
            date='NA'

        if '-' not in line[5]:
            beam=np.log10(float(line[5]))
        else:
            beams=line[5].split('-')
            beamavg=(float(beams[0])+float(beams[1]))/2
            oldbeam=f'{np.log10(float(beams[0]))} - {np.log10(float(beams[1]))}'
            beam=[beamavg,oldbeam]

        if flux_upperb==0:
            sbeamsizes.append(beam)
            sfluxs.append([flux,fluxunc])
            sfreqs.append(freq)
            sdates.append(date)
            sfiles.append(filename)
            

        if flux_upperb==1:
            sbeamsizes2.append(beam)
            sfluxs2.append(flux)
            sfreqs2.append(freq)
            sdates2.append(date)
            sfiles2.append(filename)

        oldsources=copy.deepcopy(sources)

        if j2==jfin:
            source=line[0]
            fig, ax = pylab.subplots(dpi=300)
            cm2 = pylab.get_cmap('jet')   
            sNorm = matplotlib.colors.Normalize(vmin=minbeam, vmax=maxbeam)
            scalarMap2 = cmx.ScalarMappable(norm=sNorm, cmap=cm2)

            def assign_colors(beamsizes):
                colors = []
                for size in beamsizes:
                    if isinstance(size, list):
                        size = size[0]
                    colors.append(scalarMap2.to_rgba(size))  # Use colormap for valid sizes
                return colors
            
            colors2 = assign_colors(sbeamsizes2)
            colors = assign_colors(sbeamsizes)

            if len(sfreqs2)!=0:
                pylab.scatter(sfreqs2,sfluxs2, c=colors2,marker='v',s=msize) 

            if len(sfreqs)!=0:
                pylab.scatter(sfreqs,sfluxs, c=colors,s=msize)

            colorbar1 = fig.colorbar(
                    cmx.ScalarMappable(norm=matplotlib.colors.Normalize(vmin=minbeam, vmax=maxbeam), cmap=cm2),
                    label='Beamsize ["]',
                    ax=ax,
                    ticks=stick_positions
                    )
            
            colorbar1.set_label(
                r'log$_{10}$ Beamsize ["]'
            )

            minor_tick_locations = np.arange(minbeamp, maxbeamp, 0.1)
            major_ticks = colorbar1.get_ticks()
            filtered_minor_ticks = [tick for tick in minor_tick_locations if tick not in major_ticks]
            colorbar1.ax.yaxis.set_ticks(filtered_minor_ticks, minor=True)
            
            ax = pylab.gca()
            ax.set_xlim([minfreq, maxfreq])
            ax.set_xticks(x_tick_positions)
            ax.set_xticklabels(x_tick_labels)

            ax.set_ylim([minflux, maxflux]) 
            ax.set_yticks(y_tick_positions)
            ax.set_yticklabels(y_tick_labels)

            minor_locator = MultipleLocator(0.1)
            ax.xaxis.set_minor_locator(minor_locator)
            ax.yaxis.set_minor_locator(minor_locator)
                                
            pylab.yticks(y_tick_positions, y_tick_labels)
            pylab.xlabel(r'log$_{10}$ Frequency [GHz]')
            pylab.title(f'{source}',fontsize=20)
            pylab.ylabel(r'log$_{10}$ Flux [mJy][Beam]$^{-1}$')
            nicename=f'{source} Spectra.pdf'
            if not os.path.exists(f'SpectraGraphs'):
                os.makedirs('SpectraGraphs')
            if os.path.exists(f'SpectraGraphs/{nicename}'):
                os.remove(f'SpectraGraphs/{nicename}')
            pylab.savefig(f'SpectraGraphs/{nicename}', dpi=300)
            pylab.show()
            pylab.clf() 

            if len(sfreqs)!=0:
                grouped_data[source]['sfreqs']=sfreqs
                grouped_data[source]['sfluxs']=sfluxs
                grouped_data[source]['sbeams']=sbeamsizes
                grouped_data[source]['sdates']=sdates
                grouped_data[source]['sfiles']=sfiles

                grouped_data[source]['sfreqs2']=sfreqs2
                grouped_data[source]['sfluxs2']=sfluxs2
                grouped_data[source]['sbeams2']=sbeamsizes2
                grouped_data[source]['sdates2']=sdates2
                grouped_data[source]['sfiles2']=sfiles2


In [ ]:
# functino to Group frequencies dynamically
def group_by_frequency(freqs, fluxes, beams, dates, tolerance):
    grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates = [], [], [], []
    current_group_freqs, current_group_fluxes, current_group_beams, current_group_dates = [], [], [], []

    sorted_data = sorted(zip(freqs, fluxes, beams, dates), key=lambda x: x[0])
    for freq, flux, beam, date in sorted_data:
        if not current_group_freqs or abs(freq - current_group_freqs[0]) <= tolerance:
            current_group_freqs.append(freq)
            current_group_fluxes.append(flux)
            current_group_beams.append(beam)
            current_group_dates.append(date)
        else:
            grouped_freqs.append(current_group_freqs)
            grouped_fluxes.append(current_group_fluxes)
            grouped_beams.append(current_group_beams)
            grouped_dates.append(current_group_dates)
            current_group_freqs = [freq]
            current_group_fluxes = [flux]
            current_group_beams = [beam]
            current_group_dates = [date]

    if current_group_freqs:
        grouped_freqs.append(current_group_freqs)
        grouped_fluxes.append(current_group_fluxes)
        grouped_beams.append(current_group_beams)
        grouped_dates.append(current_group_dates)

    return grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates


In [ ]:
#Define frequency ranges for VLA and ALMA bands
vla_bands = [
    (0.0,0.054) ,     #for Rounding 
    (0.054, 0.086),  # 4 Band
    (0.230, 0.470),  # P Band
    (1, 2),          # L Band
    (2, 4),          # S Band
    (4, 8),          # C Band
    (8, 12),         # X Band
    (12, 18),        # Ku Band
    (18, 26.5),      # K Band
    (26.5, 40),      # Ka Band
    (40, 50),        # Q Band
]

alma_bands = [
    (31.3, 45),   # Band 1
    (67, 90),     # Band 2
    (84, 116),    # Band 3
    (125, 163),   # Band 4
    (163, 211),   # Band 5
    (211, 275),   # Band 6
    (275, 373),   # Band 7
    (385, 500),   # Band 8
    (602, 720),   # Band 9
    (787, 950),   # Band 10
]

allfluxs=[]

# Combine all boundaries and create continuous intervals
all_boundaries = sorted(set([freq for band in vla_bands + alma_bands for freq in band]))
non_overlapping_intervals = [(all_boundaries[i], all_boundaries[i + 1]) for i in range(len(all_boundaries) - 1)]

# Remove gaps by ensuring no holes
continuous_intervals = []
for i, (start, end) in enumerate(non_overlapping_intervals):
    if i > 0 and start > continuous_intervals[-1][1]:
        # Fill the gap between the previous interval and the current one
        continuous_intervals.append((continuous_intervals[-1][1], start))
    continuous_intervals.append((start, end))

# Function to check if a frequency is in any interval
def is_frequency_in_intervals(frequency, intervals):
    for start, end in intervals:
        if start <= frequency < end:  # Check if frequency is in the interval
            return True, (start, end)
    return False, None

# Print all intervals for verification
print("\nAll continuous intervals:")
for i, interval in enumerate(continuous_intervals):
    print(f"Interval {i + 1}: {interval[0]}–{interval[1]} GHz")

def group_by_band(freqs, fluxes, beams, dates, intervals):
    grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates = [], [], [], []
    interval_mapping = {interval: ([], [], [], []) for interval in intervals}

    for freq, flux, beam, date in zip(freqs, fluxes, beams, dates):
        found, assigned_interval = is_frequency_in_intervals(freq, intervals)
        if found:
            interval_mapping[assigned_interval][0].append(freq)
            interval_mapping[assigned_interval][1].append(flux)
            interval_mapping[assigned_interval][2].append(beam)
            interval_mapping[assigned_interval][3].append(date)

    for interval, (freq_list, flux_list, beam_list, date_list) in interval_mapping.items():
        if freq_list:
            grouped_freqs.append(freq_list)
            grouped_fluxes.append(flux_list)
            grouped_beams.append(beam_list)
            grouped_dates.append(date_list)

    return grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates


In [ ]:
#Generate Statistical Spectral Plots
# Define tolerance for grouping frequencies (log space)
tolerance = .05

#define if it should be grouped dynamicall or by band
grouper='byband'
#grouper='bydynam'

# Generate consistent x-axis and y-axis tick positions
x_tick_positions = np.linspace(minfreq, maxfreq, num=6)  
x_tick_labels = [f"{10**pos:.2f}" for pos in x_tick_positions]  

y_tick_positions = np.linspace(minflux, maxflux, num=6) 
y_tick_labels = [f"{10**pos:.2f}" for pos in y_tick_positions]  

# Apply grouping to all sources
for source, data in grouped_data.items():
    if grouper=='bydynam':
        grouped_data[source]['grouped_sfreqs'], grouped_data[source]['grouped_sfluxs'], grouped_data[source]['grouped_sbeams'], grouped_data[source]['grouped_sdates'] = group_by_frequency(
            data['sfreqs'], data['sfluxs'], data['sbeams'],data['sdates'],tolerance
        )
        grouped_data[source]['grouped_sfreqs2'], grouped_data[source]['grouped_sfluxs2'], grouped_data[source]['grouped_sbeams2'], grouped_data[source]['grouped_sdates2'] = group_by_frequency(
            data['sfreqs2'], data['sfluxs2'], data['sbeams2'], data['sdates2'],tolerance
        )

    if grouper=='byband':
        grouped_data[source]['grouped_sfreqs'], grouped_data[source]['grouped_sfluxs'], grouped_data[source]['grouped_sbeams'], grouped_data[source]['grouped_sdates'] = group_by_band(
            data['sfreqs'], data['sfluxs'], data['sbeams'], data['sdates'], continuous_intervals
        )
        
        grouped_data[source]['grouped_sfreqs2'], grouped_data[source]['grouped_sfluxs2'], grouped_data[source]['grouped_sbeams2'], grouped_data[source]['grouped_sdates2'] = group_by_band(
            data['sfreqs2'], data['sfluxs2'], data['sbeams2'], data['sdates2'], continuous_intervals
        )


# Initialize combined arrays
combined_freqs = []  # Combined frequencies
combined_fluxes = []  # Combined fluxes
combined_beams = []  # Combined beam sizes

for source, data in grouped_data.items():
    # Combine all frequency, flux, and beam data for the current source
    combined_freqs = data['sfreqs'] 
    combined_fluxes = data['sfluxs'] 
    combined_fluxes = [float(f[0]) for f in combined_fluxes ]
    combined_beams = data['sbeams'] 
    combined_dates = data['sdates'] 

    combined_freqs_2 = data['sfreqs2']
    combined_fluxes_2 = data['sfluxs2'] 
    combined_beams_2 = data['sbeams2'] 
    combined_dates_2 = data['sdates2'] 

    # Group the combined data dynamically for the current source
    if grouper=='bydynam':
        grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates = group_by_frequency(
            combined_freqs, combined_fluxes, combined_beams, combined_dates, tolerance
        )

    if grouper=='byband':
        grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates = group_by_band(
            combined_freqs, combined_fluxes, combined_beams, combined_dates, continuous_intervals
        )

    # Iterate over grouped combined data
    for freq_group, flux_group, beam_group in zip(grouped_freqs, grouped_fluxes, grouped_beams):
        #unc_group  = [float(f[1]) for f in flux_group]
        group_size = len(flux_group)
        central_freq = np.mean(freq_group)
        group_size = len(flux_group)
        central_freq = np.mean(freq_group)
        if group_size >= 4:
            pylab.boxplot(flux_group, positions=[central_freq],widths=0.1,
            showfliers=False)  # Customize outliers
        elif group_size == 3:
            # Vertical line with caps at min and max
            min_flux, max_flux = min(flux_group), max(flux_group)
            median_flux = np.median(flux_group)
            pylab.errorbar(
                x=[central_freq], 
                y=[median_flux], 
                yerr=[[median_flux - min_flux], [max_flux - median_flux]], 
                capsize=5,  # Length of caps
                color='k'
            )
            # Add a horizontal line at the median_flux
            pylab.hlines(
                y=median_flux,  # Position of the horizontal line
                xmin=central_freq - 0.05,  # Adjust as needed for length
                xmax=central_freq + 0.05, 
                color='r',  # Color of the line
                linewidth=1  # Thickness of the horizontal line
            )
        elif group_size == 2:
            # Vertical line with caps at min and max (no median)
            min_flux, max_flux = min(flux_group), max(flux_group)
            median_flux = (min_flux + max_flux) / 2  # Just for centering the line
            pylab.errorbar(
                x=[central_freq], 
                y=[median_flux], 
                yerr=[[median_flux - min_flux], [max_flux - median_flux]], 
                capsize=5, 
                color='k'
            )

        elif group_size == 1:
            pylab.scatter([central_freq], [flux_group[0]], c='b', marker='o', s=10)

    if combined_fluxes_2:  # Ensure '2' fluxes exist
        if not combined_fluxes or min(combined_fluxes_2) < min(combined_fluxes):
            # If main fluxes are empty OR '2' minimum is lower
            min_flux_2 = min(combined_fluxes_2)
            min_flux_2_index = combined_fluxes_2.index(min_flux_2)
            min_freq_2 = combined_freqs_2[min_flux_2_index]
            # Mark the lowest value with a downward triangle
            pylab.scatter([min_freq_2], [min_flux_2], c='black', marker='v', s=7, label='Min Flux from 2')


    # Set consistent x and y limits for all plots
    pylab.xlim(minfreq - 0.1, maxfreq + 0.1)  # Add slight padding
    pylab.ylim(minflux - 0.1, maxflux + 0.1)  # Add slight padding
    # Set consistent x-axis ticks and labels
    # Set consistent x-axis and y-axis ticks and labels
    pylab.xticks(x_tick_positions, x_tick_labels)
    pylab.yticks(y_tick_positions, y_tick_labels)

    # Add labels and title
    pylab.xlabel('Frequency [GHz]')
    pylab.ylabel(r'Flux [mJy][Beam]$^{-1}$')
    pylab.title(f'{source}')

    nicename=f'{source} statistical spectra.pdf'
    if not os.path.exists(f'SpectraGraphs/{source}'):
        os.makedirs(f'SpectraGraphs/{source}')
    if os.path.exists(f'SpectraGraphs/{source}/{nicename}'):
        os.remove(f'SpectraGraphs/{source}/{nicename}')
    pylab.savefig(f'SpectraGraphs/{source}/{nicename}', dpi=300)
    pylab.show()
    pylab.clf() 

In [ ]:
#make function to get distances

import os
from astroquery.ned import Ned
import ast  # For safely parsing text file data

import os
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np

from astropy.coordinates import SkyCoord
import numpy as np
import astropy.units as u

import re

import requests
import csv
from scipy import constants
c=constants.c

def is_subsequence(small, large):
    it = iter(large)
    return all(char in it for char in small)

def decompose_string(s):
    parts = re.findall(r"[A-Za-z]+|\d+", s)  # Finds all letter and number sequences
    return parts if parts else [s]

# Input file with source names and coordinates
txtnamesandcoords = 'namesandcoords.txt'

# List of desired reference codes
wantrefs = ['2018ApJ...863..', '2023ApJ...948..', '2020ApJ...890..', '2020ApJ...891..']

def getdists(newnames):
    newrefs=[]
    if not os.path.exists('distancetable.csv'):
        url = 'https://ned.ipac.caltech.edu/Archive/Distances/NED30.5.1-D-17.1.2-20200415.csv'
        response = requests.get(url)

        if response.status_code == 200:
            with open('distancetable.csv', "wb") as file:
                file.write(response.content)
        else:
            return(f"Failed to download. Status code: {response.status_code}")
    
    maserdist=[]
    otherdist=[]
    j1=-1
    j2=-1
    with open('distancetable.csv', "r", newline="", encoding="utf-8") as file:
        reader = csv.reader(file)
        for row in reader:
            textsource=row[3]
            if len(textsource)==0:
                continue
            textdistmm=row[4]
            textdistmmerr=row[5]

            textdistMpc=row[6]
            textmethod=row[7]
            refcode=row[8]


            textsource=textsource.replace(' ','')

            findred=0
            for i in newnames:
                checkers=[]
                for ii in i:
                    ii=ii.split('Galaxy')[0]
                    if is_subsequence(ii, textsource):
                        checkers.append(ii)
                if len(checkers)==len(i):
                    if row[10]:
                        findred=1
                        redshift=row[10]
                        Hubble=row[11]
                        newdist=c*float(redshift)/float(Hubble)*c
                    
                    if textdistmmerr:
                        if float(textdistmmerr)>0:
                            if 'maser' in textmethod.lower():
                                j1=j1+1
                                newrefs.append(refcode)
                                if j1==0:
                                        maserdist=[textsource,textdistmm,textdistmmerr,textdistMpc,textmethod,refcode]
                                        maserdisuncer=textdistmmerr
                                elif textdistmmerr<maserdisuncer:
                                    maserdist=[textsource,textdistmm,textdistmmerr,textdistMpc,textmethod,refcode]
                            else:
                                j2=j2+1
                                if j2==0:
                                    otherdist=[textsource,textdistmm,textdistmmerr,textdistMpc,textmethod,refcode]
                                    otherdisuncer=textdistmmerr
                                elif textdistmmerr<otherdisuncer:
                                    otherdist=[textsource,textdistmm,textdistmmerr,textdistMpc,textmethod,refcode]

    return(maserdist,otherdist,refcode,findred,newrefs)

# Function to process each source
def process_source(source_name, wantrefs, ra, dec,threshold, secondarynames):
  
    """Query NED for redshift data and filter based on references and uncertainties."""
    # Skip excluded sources

    # Query the redshift table from NED
    distances_table = Ned.get_table(source_name, table='redshifts')

    # Extract relevant columns
    redshifts = distances_table['Published Redshift']
    uncertainties = distances_table['Published Redshift Uncertainty']
    refcodes = distances_table['Refcode']

    # Combine into tuples
    entries = [
        (redshift, uncertainty, refcode)
        for redshift, uncertainty, refcode in zip(redshifts, uncertainties, refcodes)
    ]

    # Filter for entries with desired references
    desired_entries = [
        entry for entry in entries if any(ref in entry[2] for ref in wantrefs)
    ]

    entries_with_reference = []
    entries_without_reference = []

    for redshift, uncertainty, refcode in desired_entries:
        if any(ref in refcode for ref in wantrefs):  # Check if refcode contains a wanted reference
            entries_with_reference.append((redshift, uncertainty, refcode))
        else:
            entries_without_reference.append((redshift, uncertainty, refcode))


    reduced_entries = {}

    # Step 2a: Prioritize entries from `entries_with_reference`
    if entries_with_reference:
        for redshift, uncertainty, refcode in entries_with_reference:
            if uncertainty > 0:  # Ignore entries with zero uncertainty
                if refcode not in reduced_entries or uncertainty < reduced_entries[refcode][1]:
                    reduced_entries[refcode] = (redshift, uncertainty)

    if len(reduced_entries)==0:
        # Step 2b: If `entries_with_reference` is empty, use `entries_without_reference`
        for redshift, uncertainty, refcode in entries_without_reference:
            if uncertainty > 0:  # Ignore entries with zero uncertainty
                if refcode not in reduced_entries or uncertainty < reduced_entries[refcode][1]:
                    reduced_entries[refcode] = (redshift, uncertainty)



    # Print results based on the filtering
    redshift='NA'
    uncertainty='NA'

    newnames=[]
    newlist=decompose_string(secondarynames[0][0])

    newnames.append(newlist)
    if len(secondarynames[0][1])!=0:
        if secondarynames[0][1][0]!='NA':
            for i in secondarynames[0][1]:
                if not is_subsequence(secondarynames[0][0], i):
                    newlist=decompose_string(i)
                    newnames.append(newlist)

    maserdists, otherdists, inewrefs, findred, newrefs = getdists(newnames)

    if reduced_entries:
        for irefcode, (iredshift, iuncertainty) in reduced_entries.items():
            redshift=iredshift
            ynfindref='y'
            refcode=irefcode
            if iuncertainty>0:
                uncertainty=iuncertainty
            else:
                input(f'No uncertainty for chosen referencecode: {refcode} , {source}')
                uncertainty='NA'
    else:

        # If no desired references, and no maser distances, find valid entries with uncertainties
        valid_entries = [entry for entry in entries if entry[1] > 0]
        if valid_entries:
            # Select the entry with the lowest uncertainty
            lowest_uncertainty_entry = min(valid_entries, key=lambda x: x[1])
            lowest_redshift, lowest_uncertainty, reference_code = lowest_uncertainty_entry
            redshift=lowest_redshift
            uncertainty= lowest_uncertainty
            refcode=reference_code
            ynfindref='n'
        else:
            input(f"No valid redshift entries found for {source_name} (1)\n")
        if len(maserdists)!=0:
            z_observed = redshift
            if z_observed >= threshold:
                if len(maserdists)!=0 and findred==1:
                        input('new maser source found... adjust accordingly (1)')
        
    z_observed = redshift
    z_uncer=uncertainty

    if z_observed <= threshold:
        if len(maserdists)!=0:
            input('new maser source found... adjust accordingly (2)')
        if len(maserdists)==0:
            textdistmm=float(otherdists[1])
            textdistmmerr=float(otherdists[2])
            textdistMpc=float(otherdists[3])

            newdist=10**((textdistmm)/5-5)
            newdist_uncer=np.log(10)/5*newdist*textdistmmerr
            textmethod=otherdists[4]
            refcode=otherdists[5]

            return([newdist,newdist_uncer],textmethod,'NA',refcode,'DISTANCE',newrefs)

    # Calculate heliocentric velocity
    c = 299792.458  # Speed of light in km/s
    v_helio = z_observed * c
    v_helio_uncer = z_uncer * c

    #Hubble Constant
    #Pesce, APJ 891:L1
    H = 73.9
    H_uncer = 3
    dist=v_helio/H
    dist_uncer=np.sqrt((v_helio_uncer/H)**2 + (v_helio/H**2*H_uncer)**2)

    return([dist,dist_uncer],'NA',ynfindref,refcode,'REDSHIFT', newrefs)

In [ ]:
#Path on local computer to SMBH mass
from pathlib import Path
parent_dir = Path.cwd().parent
Greenepath=f'{parent_dir}/CountingBHS_and_Spectra/Greene_params/Greene_params.txt'

In [ ]:
#date format
from datetime import datetime, timedelta

def decimal_year_to_date(decimal_year):
    if decimal_year == 'NA':
        return 'NA'

    year = int(decimal_year)
    yearremainder = decimal_year - year

    # First extract the month
    decmonth = yearremainder * 12 + 1
    month = int(decmonth)
    monthremainder = decmonth - month

    day = int(round(monthremainder/12*365+1))


    return f"{year}/{month:02}/{day:02}"

In [ ]:
#get SMBH size
from scipy import constants
from astropy import constants as aconstants
from matplotlib.lines import Line2D

pc=aconstants.pc.value
mass_sun=aconstants.M_sun.value
G=constants.gravitational_constant
pi=constants.pi

def getsize(log_BH_mass_solar,log_BH_mass_solar_uncer,distanceMPc,distanceMPc_uncer,arcsectopc,arcsectopc_unc):
    mass=10**float(log_BH_mass_solar)*mass_sun
    mass_unc=mass*np.log(10)*float(log_BH_mass_solar_uncer)
    mass_fracerror=(mass_unc/mass)*100

    distance=distanceMPc*pc*10**6
    distance_uncer=distanceMPc_uncer*pc*10**6
    dist_fracerror=(distance_uncer/distance)*100
    
    if arcsectopc=='NA':
        m_d=mass/distance
        m_d_unc=np.sqrt((m_d/mass*mass_unc)**2+(m_d/distance*distance_uncer)**2)
        m_d_fracerror=(m_d_unc/m_d)*100
        diam_bhs_microarcsec_fracerror = m_d_fracerror

        diam_bhs_rad=2*np.sqrt(27)*G/(c**2)*m_d
        diam_bhs_deg=diam_bhs_rad*360/(2*pi)
        diam_bhs_arcsec=diam_bhs_deg*60*60
        diam_bhs_microarcsec=diam_bhs_arcsec*10**6

        diam_bhs_microarcsec_unc=diam_bhs_microarcsec/(m_d)*m_d_unc


        return(log_BH_mass_solar, log_BH_mass_solar_uncer, mass_fracerror, distanceMPc, distanceMPc_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_unc, diam_bhs_microarcsec_fracerror)

        
    diam_bhs=2*np.sqrt(27)*G/(c**2)*mass
    diam_bhs_unc = 2*np.sqrt(27)*G/(c**2)*mass_unc
    
    diam_bhspc=diam_bhs/pc
    diam_bhspc_unc=diam_bhs_unc/pc

    diam_bhs_arsec=diam_bhspc/arcsectopc
    diam_bhs_microarcsec=diam_bhs_arsec*10**6

    diam_bhs_arsec_unc=np.sqrt((diam_bhspc_unc/arcsectopc)**2
                               + ((-1)*diam_bhspc/arcsectopc**2*arcsectopc_unc)**2
                               )
    
    diam_bhs_microarcsec_unc=diam_bhs_arsec_unc*10**6
    diam_bhs_microarcsec_fracerror = (diam_bhs_microarcsec_unc/diam_bhs_microarcsec)*100

    return(mass, mass_unc, mass_fracerror, distance, distance_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_unc, diam_bhs_microarcsec_fracerror)

sagmass=6.6
sagmass_unc_up=.12
sagmass_unc_down=.07
sagdist=8.15*10**(-3)
sagdist_unc=.15*10**(-3)

distanceMPc=sagdist
distanceMPc_uncer=sagdist_unc

log_BH_mass_solar=sagmass
log_BH_mass_solar_uncerup=sagmass_unc_up
log_BH_mass_solar_uncerdown=sagmass_unc_down

mass, mass_uncup, mass_fracerrorup, distance, distance_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_uncup, diam_bhs_microarcsec_fracerrorup=getsize(sagmass,sagmass_unc_up,sagdist,sagdist_unc,'NA','NA')
mass, mass_uncdown, mass_fracerrordown, distance, distance_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_uncdown, diam_bhs_microarcsec_fracerrordown=getsize(sagmass,sagmass_unc_down,sagdist,sagdist_unc,'NA','NA')

diam_bhs_microarcsec_uncup=format_number(diam_bhs_microarcsec_uncup)
diam_bhs_microarcsec_uncdown=format_number(diam_bhs_microarcsec_uncdown)
diam_bhs_microarcsec_fracerror=(diam_bhs_microarcsec_fracerrorup+diam_bhs_microarcsec_fracerrordown)/2

sourcehan=Line2D([0], [0], color='red', marker='o', markersize=0, label=f'SagA*', linestyle='')
sizechan=Line2D([0], [0], color='red', marker='o', markersize=0, label=rf'µ" of Scharchild Radius: {format_number(diam_bhs_microarcsec)} $^{{+{diam_bhs_microarcsec_uncup}}}_{{-{diam_bhs_microarcsec_uncdown}}}$ ({format_number(diam_bhs_microarcsec_fracerror)}%)', linestyle='')
disthan=Line2D([0], [0], color='red', marker='o', markersize=0, label=f'Distance From Earth: {format_number(distanceMPc)} ± {format_number(distanceMPc_uncer)} MPc ({format_number(dist_fracerror)}%)', linestyle='')
log_BH_mass_solar_uncerup=format_number(float(log_BH_mass_solar_uncerup))
log_BH_mass_solar_uncerdown=format_number(float(log_BH_mass_solar_uncerdown))
mass_fracerror=(mass_fracerrorup+mass_fracerrordown)/2
masshan=Line2D([0], [0], color='red', marker='o', markersize=0, label=rf'SMBH Mass: {format_number(float(log_BH_mass_solar))} $^{{+{log_BH_mass_solar_uncerup}}}_{{-{log_BH_mass_solar_uncerdown}}}$ log M☉ ({format_number(mass_fracerror)}%)', linestyle='')
handles1=[sourcehan,sizechan,masshan,disthan]

fig, ax = pylab.subplots(dpi=300)

m87mass=9.81
m87mass_unc=0.06
m87dist=16.8
m87dist_uncup=.8
m87dist_uncdown=.7

distanceMPc=m87dist
distanceMPc_uncerup=m87dist_uncup
distanceMPc_uncerdown=m87dist_uncdown

log_BH_mass_solar=m87mass
log_BH_mass_solar_uncer=m87mass_unc

mass, mass_unc, mass_fracerror, distance, distance_uncerup, dist_fracerrorup, diam_bhs_microarcsec, diam_bhs_microarcsec_uncup, diam_bhs_microarcsec_fracerrorup=getsize(m87mass,m87mass_unc,m87dist,m87dist_uncup,'NA','NA')
mass, mass_unc, mass_fracerror, distance, distance_uncerdown, dist_fracerrordown, diam_bhs_microarcsec, diam_bhs_microarcsec_uncdown, diam_bhs_microarcsec_fracerrordown=getsize(m87mass,m87mass_unc,m87dist,m87dist_uncdown,'NA','NA')

diam_bhs_microarcsec_uncup=format_number(diam_bhs_microarcsec_uncup)
diam_bhs_microarcsec_uncdown=format_number(diam_bhs_microarcsec_uncdown)
diam_bhs_microarcsec_fracerror=(diam_bhs_microarcsec_fracerrorup+diam_bhs_microarcsec_fracerrordown)/2

sourcehan=Line2D([0], [0], color='red', marker='o', markersize=0, label=f'M87*', linestyle='')
sizechan=Line2D([0], [0], color='red', marker='o', markersize=0, label=rf'µ" of Scharchild Radius: {format_number(diam_bhs_microarcsec)} $^{{+{diam_bhs_microarcsec_uncup}}}_{{-{diam_bhs_microarcsec_uncdown}}}$ ({format_number(diam_bhs_microarcsec_fracerror)}%)', linestyle='')
dist_fracerror=(distanceMPc_uncerup+distanceMPc_uncerdown)/2
distanceMPc_uncerup=format_number(float(distanceMPc_uncerup))
distanceMPc_uncerdown=format_number(float(distanceMPc_uncerdown))
disthan=Line2D([0], [0], color='red', marker='o', markersize=0, label=f'Distance From Earth: {format_number(distanceMPc)} $^{{+{distanceMPc_uncerup}}}_{{-{distanceMPc_uncerdown}}}$ MPc ({format_number(dist_fracerror)}%)', linestyle='')
masshan=Line2D([0], [0], color='red', marker='o', markersize=0, label=rf'SMBH Mass: {format_number(float(log_BH_mass_solar))} ±{log_BH_mass_solar_uncerdown} log M☉ ({format_number(mass_fracerror)}%)', linestyle='')
handles2=[sourcehan,sizechan,masshan,disthan]

legend1=ax.legend(handles=handles2, loc='upper center')   
ax.add_artist(legend1)
ax.legend(handles=handles1, loc='lower center')   
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_xticks([])  # Remove x-axis ticks
ax.set_yticks([])  # Remove y-axis ticks

# Remove tick labels
ax.set_xticklabels([])  # Remove x-axis labels
ax.set_yticklabels([])  # Remove y-axis labels
if not os.path.exists('Extras'):
    os.makedirs('Extras')
pylab.savefig(f'Extras/SMBH of SagA* and M87')
pylab.clf() 


In [ ]:
#Print All Correlations
write=True
#write=False
printall=True
#colorbarvar='time'
#for variability avg val is flux and for compactness it is beam
#Actually it will give the lower value
colorbarvar='avgval'

def percentage_difference(x, y):
    return (x - y) / ((x + y) / 2) * 100

def percentage_difference_unc(x, y, x_unc, y_unc):
    
    term1 = 4*y/(x+y)**2 * x_unc * 100
    term2 = 4*x/(x+y)**2 * y_unc * 100

    return np.sqrt(term1**2 + term2**2)

import ast
from matplotlib.lines import Line2D
from scipy import constants
from astropy import constants as aconstants
import sys
import matplotlib.pyplot as pylab
import os
import shutil

allfluxs=[]

def format_val(val):
    if val==0:
        return '0'
    if np.isnan(val):
        return np.nan
    places = max(1,  -int(math.floor(math.log10(abs(val))))+1) 
    #return f'{val:.{places}f}'
    if val >1:
        val=f'{val:.1f}' 
    elif val >0.1:
        val=f'{val:.2f}' 
    elif val >0.01:
        val=f'{val:.3f}'
    elif val >0.001:
        val=f'{val:.4f}'
    else:
        val=f"{val:.{places}f}"
    return val

def format_val2(val):
    if val==0:
        return '0'
    if val<0.1:
        val=f'{val:.4f}'
    elif val<1:
        val=f'{val:.3f}'
    elif val<10:
        val=f'{val:.2f}'
    elif val<100:
        val=f'{val:.1f}'
    else:
        input(f'problem with beam {val}')
    return val

cm2 = pylab.get_cmap('jet')  

def round_down_to_half_integer(num):
    return math.floor(num * 2) / 2

def round_up_to_half_integer(num):
    return math.ceil(num * 2) / 2

pc=aconstants.pc.value
mass_sun=aconstants.M_sun.value
c=constants.c
G=constants.gravitational_constant
pi=constants.pi

threshold=0.0015

txtnamesandcoords='namesandcoords.txt'
path='Beam Mismatch'
if os.path.exists(path):
    shutil.rmtree(path)
os.mkdir(path)

spacings=[13, 8, 8, 10, 9, 9, 10, 10, 10, 15, 15, 15, 10]
tot = 0
for i in spacings:
    tot=tot+i
source='target'

sfreqs1='freq1'
sfreqs2='freq2'
freqper='[sep %]'

sbeams1='beam1'
sbeams2='beam2'
beamper='[sep %]'

sfluxes1='flux1'
sfluxes2='flux2'
fluxper='[sep %]'

sdates1='year1'
sdates2='year2'
datesep='[sep yrs]'

if printall==True:
    corr_table='machinetables/correlations_machine.txt'

with open(corr_table, 'w') as file:
    if write==True:
        original_stdout = sys.stdout
        sys.stdout = file
    print(f'{source.ljust(spacings[0])} {sfreqs1.ljust(spacings[1])} {sfreqs2.ljust(spacings[2])} {freqper.ljust(spacings[3])} {sbeams1.ljust(spacings[4])} {sbeams2.ljust(spacings[5])} {beamper.ljust(spacings[6])} {sfluxes1.ljust(spacings[7])} {sfluxes2.ljust(spacings[8])} {fluxper.ljust(spacings[9])} {sdates1.ljust(spacings[10])} {sdates2.ljust(spacings[11])} {datesep.ljust(spacings[12])}')

    newrefs=[]
    indextot=-1
    for isource, data in grouped_data.items():
        if isource!='NGC1194':
            #continue
            pass
        # Main loop to read and process sources from the input file
        if os.path.exists(txtnamesandcoords):
            with open(txtnamesandcoords, 'r') as infile:
                for line in infile:
                    # Parse the line into a list of name-coordinate pairs
                    params = ast.literal_eval(line)
                    for inamecoords in params:              
                        source_name = inamecoords[0][0]
                        if not isource.lower() in source_name.lower():
                            continue
                        source=source_name
                        if source=='CircinusGalaxy':
                            source='Circinus'    
                        ra = inamecoords[1][1]
                        dec = inamecoords[1][2]  # Galaxy Dec in sexagesimal format
                        findvals=process_source(source_name, wantrefs, ra, dec, threshold, inamecoords)
                        distanceMPc=findvals[0][0]
                        distanceMPc_uncer=findvals[0][1]
                        method=findvals[1]
                        ynfindref=findvals[2]
                        refcode=findvals[3]
                        red_depend=findvals[4]
                        if len(findvals[5])!=0:
                            for i in findvals[5]:
                                newrefs.append(i)
        else:
            print(f"File {txtnamesandcoords} does not exist.")

        arcsectokpc= np.pi / (180 * 3600) * distanceMPc * 1000
        arcsectopc=arcsectokpc*1000

        arcsectokpc_error = np.pi / (180 * 3600) * distanceMPc_uncer * 1000
        arcsectopc_unc=arcsectokpc_error*1000

        arcsectopc_fracerror=(arcsectopc_unc/arcsectopc)*100

        distance='NA'
        with open(Greenepath, 'r') as infile:
            j=-1
            breaker=1
            for line in infile:
                j=j+1
                if j>20:
                    continue
                source_name=line.split(' ')[0]
                source_name=source_name.replace("−","-")
                if "WISEA" in isource:
                    isource=isource.split("WISEA")[0]
                if isource.lower() in source_name.lower():
                    log_BH_mass_solar=line.split(' ')[3]
                    log_BH_mass_solar_offset=line.split(' ')[5]
                    breaker=0
            if breaker!=1:
                mass, mass_unc, mass_fracerror, distance, distance_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_unc, diam_bhs_microarcsec_fracerror=getsize(log_BH_mass_solar,log_BH_mass_solar_uncer,distanceMPc,distanceMPc_uncer,arcsectopc,arcsectopc_unc)

        # Filter out elements where date == 'NA'
        valid_sfreqs = [(f, fl, b, d) for f, fl, b, d in zip(data['sfreqs'], data['sfluxs'], data['sbeams'], data['sdates'])]
        valid_sfreqs2 = [(f, fl, b, d) for f, fl, b, d in zip(data['sfreqs2'], data['sfluxs2'], data['sbeams2'], data['sdates2'])]

        # Combine all valid data for grouping
        combined_data = valid_sfreqs + valid_sfreqs2
        
        # Unpack valid data
        if combined_data:
            combined_freqs, combined_fluxes, combined_beams, combined_dates = zip(*combined_data)
        else:
            combined_freqs, combined_fluxes, combined_beams, combined_dates = [], [], [], []

        grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates = [list(combined_freqs)], [list(combined_fluxes)], [list(combined_beams)], [list(combined_dates)]

        #print(grouped_fluxes)
        # Iterate through frequency groups and create separate plots
        donefreq=[]

        filtered_freqs='NA'

        for freq_group, flux_group, beam_group, date_group in zip(grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates):
            if len(date_group) > 0 and len(flux_group) > 0:  # Only plot if there are valid points  

                new_valid=[]
    
                for ivalid_sfreqs in valid_sfreqs:
                    ivalid_sfreqs=list(ivalid_sfreqs)
                    ivalid_sfreqs.append('detect')
                    ivalid_sfreqs=tuple(ivalid_sfreqs)
                    new_valid.append(ivalid_sfreqs)

                for ivalid_sfreqs in valid_sfreqs2:
                    ivalid_sfreqs=list(ivalid_sfreqs)
                    ivalid_sfreqs.append('nodetect')
                    ivalid_sfreqs=tuple(ivalid_sfreqs)
                    new_valid.append(ivalid_sfreqs)


                #for valid_data in new_valid:
                #for valid_data in [
                #    (valid_sfreqs)
                #]:  
                    #filtered_points = [(d, fl, b, f) for f, fl, b, d in valid_data if f in freq_group]


                for valid_data in [new_valid]:
                    filtered_points = [(d, fl, b, f, detect) for f, fl, b, d, detect in valid_data if f in freq_group]
                    if filtered_points:

                        filtered_dates, filtered_fluxes, filtered_beams, filtered_freqs, detect = zip(*filtered_points)

                        #filtered_dates, filtered_fluxes, filtered_beams, filtered_freqs = zip(*filtered_points)
                        
                        #for getting into MPc values
                        #filtered_beams = np.array(filtered_beams)
                        #filtered_beams = 10**filtered_beams
                        #filtered_beams = filtered_beams * (np.pi / (180 * 3600))
                        #filtered_beams = filtered_beams * distanceMPc * 1000
                        #filtered_beams = np.log10(filtered_beams)
                        #filtered_beams = filtered_beams.tolist()

                        #filtered_fluxes=10**np.array(filtered_fluxes) * (np.pi / (180 * 3600)) * distanceMPc * 1000
                        #filtered_fluxes=filtered_fluxes.tolist()

                # Assuming filtered_freqs is already defined and is a list or array
                n = len(filtered_freqs)

                freq_diffs = np.full((n, n), np.nan, dtype=object)
                avgfreqij = np.full((n, n), np.nan, dtype=object)
                beam_diffs = np.full((n, n), np.nan, dtype=object)
                avgbeamij = np.full((n, n), np.nan, dtype=object)
                flux_diffs = np.full((n, n), np.nan, dtype=object)
                avgfluxij = np.full((n, n), np.nan, dtype=object)
                date_diffs = np.full((n, n), np.nan, dtype=object)
                detect_diffs = np.full((n, n), np.nan, dtype=object)

                for i in range(len(filtered_freqs)):
                    for j in range(i + 1, len(filtered_freqs)):
                        keep=1
                        if detect[i]=='nodetect' and detect[j]=='nodetect':
                            keep=0
                        elif detect[i]=='nodetect':
                            if filtered_fluxes[i]> filtered_fluxes[j][0]:
                                keep=0
                        elif detect[j]=='nodetect':
                            if filtered_fluxes[j] > filtered_fluxes[i][0]:
                                keep=0

                        if keep==0:
                            freq_diffs[i, j] = percentage_difference(10**filtered_freqs[i], 10**filtered_freqs[j])
                            avgfreqij[i, j]= (10**filtered_freqs[i] + 10**filtered_freqs[j])/2

                            if isinstance(filtered_beams[i], list):
                                bothi=filtered_beams[i][1].split(' - ')
                                beami1=float(bothi[0])
                                beami2=float(bothi[1])
                                beamdiff1=percentage_difference(beami1, float(filtered_beams[j]))
                                beamdiff2=percentage_difference(beami2, float(filtered_beams[j]))
                                beam_diffs[i, j] = f"{beamdiff1}-{beamdiff2}"
                            elif isinstance(filtered_beams[j], list):
                                input(filtered_beams[i])
                                bothj=filtered_beams[j][1].split(' - ')
                                beamj1=float(bothj[0])
                                beamj2=float(bothj[1])
                                beamdiff1=percentage_difference(float(filtered_beams[i]), beamj1)
                                beamdiff2=percentage_difference(float(filtered_beams[i]), beamj2)
                                beam_diffs[i, j] = f"{beamdiff1}-{beamdiff2}"
                            else:
                                beam_diffs[i, j] = percentage_difference(10**filtered_beams[i], 10**filtered_beams[j])


                            if isinstance(filtered_beams[i], list) or isinstance(filtered_beams[j], list):
                                avgbeamij[i, j]='NA'
                            else:
                                if filtered_beams[i] < filtered_beams[j]:
                                    avgbeamij[i, j]= 10**filtered_beams[i]
                                else:
                                    avgbeamij[i, j]= 10**filtered_beams[j]


                            flux_diffs[i, j] ='NA'
                            avgfluxij[i, j] ='NA'

                            if filtered_dates[i]!='NA' and filtered_dates[j]!='NA':
                                date_diffs[i, j] = np.abs(filtered_dates[i]-filtered_dates[j])
                            else:
                                date_diffs[i, j] ='NA'

                        if keep==1:
                            freq_diffs[i, j] = percentage_difference(10**filtered_freqs[i], 10**filtered_freqs[j])
                            avgfreqij[i, j]= (10**filtered_freqs[i] + 10**filtered_freqs[j])/2



                            if isinstance(filtered_beams[i], list):
                                bothi=filtered_beams[i][1].split(' - ')
                                beami1=float(bothi[0])
                                beami2=float(bothi[1])
                                beamdiff1=percentage_difference(beami1, float(filtered_beams[j]))
                                beamdiff2=percentage_difference(beami2, float(filtered_beams[j]))
                                beam_diffs[i, j] = f"{beamdiff1}-{beamdiff2}"
                            elif isinstance(filtered_beams[j], list):
                                bothj=filtered_beams[j][1].split(' - ')
                                beamj1=float(bothj[0])
                                beamj2=float(bothj[1])
                                beamdiff1=percentage_difference(float(filtered_beams[i]), beamj1)
                                beamdiff2=percentage_difference(float(filtered_beams[i]), beamj2)
                                beam_diffs[i, j] = f"{beamdiff1}-{beamdiff2}"
                            else:
                                beam_diffs[i, j] = percentage_difference(10**filtered_beams[i], 10**filtered_beams[j])

                            #avgbeamij[i, j]= (10**filtered_beams[i] + 10**filtered_beams[j])/2
                            if isinstance(filtered_beams[i], list) or isinstance(filtered_beams[j], list):
                                avgbeamij[i, j]='NA'
                            else:
                                if filtered_beams[i] < filtered_beams[j]:
                                    avgbeamij[i, j]= 10**filtered_beams[i]
                                else:
                                    avgbeamij[i, j]= 10**filtered_beams[j]
                                    
                            if isinstance(filtered_fluxes[i], list) and isinstance(filtered_fluxes[j], list):
                                flux1=10**filtered_fluxes[i][0]
                                flux2=10**filtered_fluxes[j][0]
                                flux1_unc=log_to_linear_uncertainty(filtered_fluxes[i][0],filtered_fluxes[i][1])
                                flux2_unc=log_to_linear_uncertainty(filtered_fluxes[j][0],filtered_fluxes[j][1])
                                flux_diffs[i, j] = [percentage_difference(flux1,flux2),percentage_difference_unc(flux1,flux2,flux1_unc,flux2_unc)]
                                avgfluxij[i, j] = (flux1+flux2)/2 
                            elif isinstance(filtered_fluxes[i], list):
                                flux1=10**filtered_fluxes[i][0]
                                flux2=10**filtered_fluxes[j]
                                flux1_unc=log_to_linear_uncertainty(filtered_fluxes[i][0],filtered_fluxes[i][1])
                                flux2_unc=0
                                flux_diffs[i, j] = [percentage_difference(flux1,flux2)+percentage_difference_unc(flux1,flux2,flux1_unc,flux2_unc)]
                                avgfluxij[i, j] = (flux1+flux2)/2 
                            elif isinstance(filtered_fluxes[j], list):
                                flux1=10**filtered_fluxes[i]
                                flux2=10**filtered_fluxes[j][0]
                                flux1_unc=0
                                flux2_unc=log_to_linear_uncertainty(filtered_fluxes[j][0],filtered_fluxes[j][1])
                                flux_diffs[i, j] = [percentage_difference(flux1,flux2)+percentage_difference_unc(flux1,flux2,flux1_unc,flux2_unc)]
                                avgfluxij[i, j] = (flux1+flux2)/2 

                            if filtered_dates[i]!='NA' and filtered_dates[j]!='NA':
                                date_diffs[i, j] = np.abs(filtered_dates[i]-filtered_dates[j])
                            else:
                                date_diffs[i, j] = 'NA'

        alldiffs=[]
        allfluxavg=[]

        allbeams=[]
        allbeamavg=[]

        allfreqs=[]
        alldates=[]

        nalldiffs=[]
        nallfluxavg=[]

        nallbeams=[]
        nallbeamavg=[]

        nallfreqs=[]   

        totnum=-1

        if filtered_freqs=='NA':
            continue
    
        for i in range(len(filtered_freqs)):
            for j in range(i + 1, len(filtered_freqs)):

                totnum += 1
                if date_diffs[i, j]=='NA' and colorbarvar=='time':
                    nalldiffs.append(flux_diffs[i, j])
                    nallfluxavg.append(avgfluxij[i, j])

                    nallbeams.append(beam_diffs[i, j])
                    nallbeamavg.append(avgbeamij[i, j])

                    newfreq=(10**filtered_freqs[i] + 10**filtered_freqs[j])/2
                    nallfreqs.append(newfreq)

                    sfreqs1=10**filtered_freqs[i]
                    if sfreqs1<10:
                        sfreqs1=f'{sfreqs1:.1f}'
                    else:
                        sfreqs1=f'{sfreqs1:.0f}'

                    sfreqs2=10**filtered_freqs[j]
                    if sfreqs2<10:
                        sfreqs2=f'{sfreqs2:.1f}'
                    else:
                        sfreqs2=f'{sfreqs2:.0f}'

                    if freq_diffs[i, j]<1:
                        freqper='[<1]'
                    else:
                        freqper='['+format_val(freq_diffs[i, j])+']'

                    beam1=10**filtered_beams[i]
                    beam2=10**filtered_beams[j]
                    sbeams1=format_val2(beam1)
                    sbeams2=format_val2(beam2)

                    if beam_diffs[i,j]<1:
                        beamper='[<1]'
                    else:
                        beamper='['+format_val(beam_diffs[i, j])+']'

                    if detect[i]=='nodetect':
                        fluxes1=f'{10**filtered_fluxes[i]:.1f}*'
                    else:
                        fluxes1=f'{10**filtered_fluxes[i]:.1f}'

                    if detect[j]=='nodetect':
                        fluxes2=f'{10**filtered_fluxes[j]:.1f}*'
                    else:
                        fluxes2=f'{10**filtered_fluxes[j]:.1f}'

                    sfluxes1=f'{fluxes1}'
                    sfluxes2=f'{fluxes2}'
                    
                    if flux_diffs[i, j] == 'NA':
                        fluxper = '[NA]'
                    else:
                        if flux_diffs[i, j][0]<1:
                            fluxper='[<1]'
                        else:
                            if len(flux_diffs[i, j])==2:
                                fluxper='['+format_val(flux_diffs[i, j][0]) + '±' + format_val(flux_diffs[i, j][1]) + ']'
                            else:
                                fluxper='[>'+format_val(flux_diffs[i, j][0])+']'

                    if flux_diffs[i, j]!=0:
                        allfluxs.append(flux_diffs[i, j])

                    if filtered_dates[i] =='NA' and filtered_dates[j] =='NA':
                        sdates1='[NA]'
                        sdates2='[NA]'
                    elif filtered_dates[i] =='NA':
                        sdates1='[NA]'
                        sdates2=f'{decimal_year_to_date(filtered_dates[j])}'
                    elif filtered_dates[j] =='NA':
                        sdates1=f'{decimal_year_to_date(filtered_dates[i])}'
                        sdates2='[NA]'
                    datesep='[NA]'
                    #print(f'{source.ljust(spacings[0])} {sfreqs1.ljust(spacings[1])} {sfreqs2.ljust(spacings[2])} {freqper.ljust(spacings[3])} {sbeams1.ljust(spacings[4])} {sbeams2.ljust(spacings[5])} {beamper.ljust(spacings[6])} {sfluxes1.ljust(spacings[7])} {sfluxes2.ljust(spacings[8])} {fluxper.ljust(spacings[9])} {sdates1.ljust(spacings[10])} {sdates2.ljust(spacings[11])} {datesep.ljust(spacings[12])}')
                else:
                    alldiffs.append(flux_diffs[i, j])
                    allfluxavg.append(avgfluxij[i, j])

                    allbeams.append(beam_diffs[i, j])
                    allbeamavg.append(avgbeamij[i, j])

                    alldates.append(date_diffs[i, j])
                    newfreq=(10**filtered_freqs[i] + 10**filtered_freqs[j])/2
                    allfreqs.append(newfreq)

                    sfreqs1=10**filtered_freqs[i]
                    if sfreqs1<10:
                        sfreqs1=f'{sfreqs1:.1f}'
                    else:
                        sfreqs1=f'{sfreqs1:.0f}'

                    sfreqs2=10**filtered_freqs[j]
                    if sfreqs2<10:
                        sfreqs2=f'{sfreqs2:.1f}'
                    else:
                        sfreqs2=f'{sfreqs2:.0f}'
                        
                    if freq_diffs[i, j]<1:
                        freqper='[<1]'
                    else:
                        freqper='['+format_val(freq_diffs[i, j])+']'

                    if isinstance(filtered_beams[i], list):
                        bothi=filtered_beams[i][1].split(' - ')
                        beami1=10**float(bothi[0])
                        beami2=10**float(bothi[1])
                        sbeams1=f"{format_val2(beami1)}-{format_val2(beami2)}"
                    else:
                        beam1=10**filtered_beams[i]
                        sbeams1=format_val2(beam1)


                    if isinstance(filtered_beams[j], list):
                        bothj=filtered_beams[j][1].split(' - ')
                        beamj1=10**float(bothj[0])
                        beamj2=10**float(bothj[1])
                        sbeams2=f"{format_val2(beamj1)}-{format_val2(beamj2)}"
                    else:
                        beam2=10**filtered_beams[j]
                        sbeams2=format_val2(beam2)

                    if isinstance(beam_diffs[i, j], float):
                        if beam_diffs[i, j]<1:
                            beamper='[<1]'
                        else:
                            beamper='['+format_val(beam_diffs[i, j])+']'
                    else:
                        beamper='[NA]'

                    if detect[i]=='nodetect':
                        fluxes1=f'{10**filtered_fluxes[i]:.1f}*'
                    else:
                        fluxes1=f'{10**filtered_fluxes[i][0]:.1f}'

                    if detect[j]=='nodetect':
                        fluxes2=f'{10**filtered_fluxes[j]:.1f}*'
                    else:
                        fluxes2=f'{10**filtered_fluxes[j][0]:.1f}'
                    sfluxes1=f'{fluxes1}'
                    sfluxes2=f'{fluxes2}'

                    if flux_diffs[i, j]!=0:
                        allfluxs.append(flux_diffs[i, j])

                    #print(filtered_dates[i])
                    #print(filtered_dates[j])
                    sdates1=f'{decimal_year_to_date(filtered_dates[i])}'
                    sdates2=f'{decimal_year_to_date(filtered_dates[j])}'
                    if date_diffs[i, j]!='NA':
                        idate_diffs=f"{date_diffs[i, j]:.3f}"
                    else:
                        idate_diffs='NA'
                    datesep=f'['+idate_diffs+']'
                    if flux_diffs[i, j] == 'NA':
                        fluxper = '[NA]'
                    else:
                        if flux_diffs[i, j][0]<1:
                            fluxper='[<1]'
                        else:
                            if len(flux_diffs[i, j])==2:
                                fluxper='['+format_val(flux_diffs[i, j][0]) + '±' + format_val(flux_diffs[i, j][1]) + ']'
                            else:
                                fluxper='[>'+format_val(flux_diffs[i, j][0])+']'
                    '''
                    print(sfreqs1)
                    print(sfreqs2)
                    print(freqper)
                    print(sbeams1)
                    print(sbeams2)
                    print(beamper)
                    print(sfluxes1)
                    print(sfluxes2)
                    print(fluxper)
                    print(sdates1)
                    print(sdates2)  
                    print(datesep)
                    '''
                    sbeams2=format_val2(beam2)
        
                if printall==True:
                    indextot+=1 
                    print(f'{isource.ljust(spacings[0])} {sfreqs1.ljust(spacings[1])} {sfreqs2.ljust(spacings[2])} {freqper.ljust(spacings[3])} {sbeams1.ljust(spacings[4])} {sbeams2.ljust(spacings[5])} {beamper.ljust(spacings[6])} {sfluxes1.ljust(spacings[7])} {sfluxes2.ljust(spacings[8])} {fluxper.ljust(spacings[9])} {sdates1.ljust(spacings[10])} {sdates2.ljust(spacings[11])} {datesep.ljust(spacings[12])}')

    if write==True:
        sys.stdout = original_stdout


In [ ]:
#Get Limits for Variability Search

import os

freqtot=[0,1000]

#flux_diff_threshold = 0  
freq_sim_threshold = 1.4  # frequency differences less than 5%
beam_sim_threshold = 2.6  # beam size differences less than 5%
freqthresh=True
beamthresh=True
fluxthresh=False
printall=False
    
def percentage_difference(x, y):
    return np.abs((x - y) / ((x + y) / 2)) * 100


import ast
from matplotlib.lines import Line2D
from scipy import constants
from astropy import constants as aconstants


totdateslim=[]
totfreqslim=[]
totdiffslimvar=[]
totdiffslimbeam=[]


def round_down_to_half_integer(num):
    return math.floor(num * 2) / 2

def round_up_to_half_integer(num):
    return math.ceil(num * 2) / 2

pc=aconstants.pc.value
mass_sun=aconstants.M_sun.value
c=constants.c
G=constants.gravitational_constant
pi=constants.pi

threshold=0.0015

txtnamesandcoords='namesandcoords.txt'
path='Beam Mismatch'
if os.path.exists(path):
    shutil.rmtree(path)
os.mkdir(path)

newrefs=[]
for isource, data in grouped_data.items():
    # Main loop to read and process sources from the input file
    if os.path.exists(txtnamesandcoords):
        with open(txtnamesandcoords, 'r') as infile:
            for line in infile:

                # Parse the line into a list of name-coordinate pairs
                params = ast.literal_eval(line)
                for inamecoords in params:                
                    source_name = inamecoords[0][0]
                    if not isource.lower() in source_name.lower():
                        continue
                    source=source_name
                    ra = inamecoords[1][1]
                    dec = inamecoords[1][2]  # Galaxy Dec in sexagesimal format
                    findvals=process_source(source_name, wantrefs, ra, dec, threshold, inamecoords)
                    distanceMPc=findvals[0][0]
                    distanceMPc_uncer=findvals[0][1]
                    method=findvals[1]
                    ynfindref=findvals[2]
                    refcode=findvals[3]
                    red_depend=findvals[4]
                    if len(findvals[5])!=0:
                        for i in findvals[5]:
                            newrefs.append(i)
    else:
        print(f"File {txtnamesandcoords} does not exist.")

    arcsectokpc= np.pi / (180 * 3600) * distanceMPc * 1000
    arcsectopc=arcsectokpc*1000

    arcsectokpc_error = np.pi / (180 * 3600) * distanceMPc_uncer * 1000
    arcsectopc_unc=arcsectokpc_error*1000

    arcsectopc_fracerror=(arcsectopc_unc/arcsectopc)*100

    distance='NA'
    with open(Greenepath, 'r') as infile:
        j=-1
        breaker=1
        for line in infile:
            j=j+1
            if j>20:
                continue
            source_name=line.split(' ')[0]
            source_name=source_name.replace("−","-")
            if "WISEA" in isource:
                isource=isource.split("WISEA")[0]
            if isource.lower() in source_name.lower():
                log_BH_mass_solar=line.split(' ')[3]
                log_BH_mass_solar_offset=line.split(' ')[5]
                breaker=0
        if breaker!=1:
            mass, mass_unc, mass_fracerror, distance, distance_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_unc, diam_bhs_microarcsec_fracerror=getsize(log_BH_mass_solar,log_BH_mass_solar_uncer,distanceMPc,distanceMPc_uncer,arcsectopc,arcsectopc_unc)



    # Filter out elements where date == 'NA'
    valid_sfreqs = [(f, fl, b, d) for f, fl, b, d in zip(data['sfreqs'], data['sfluxs'], data['sbeams'], data['sdates'])]
    valid_sfreqs2 = [(f, fl, b, d) for f, fl, b, d in zip(data['sfreqs2'], data['sfluxs2'], data['sbeams2'], data['sdates2'])]

    # Combine all valid data for grouping
    combined_data = valid_sfreqs + valid_sfreqs2
    
    # Unpack valid data
    if combined_data:
        combined_freqs, combined_fluxes, combined_beams, combined_dates = zip(*combined_data)
    else:
        combined_freqs, combined_fluxes, combined_beams, combined_dates = [], [], [], []

    grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates = [list(combined_freqs)], [list(combined_fluxes)], [list(combined_beams)], [list(combined_dates)]

    #print(grouped_fluxes)
    # Iterate through frequency groups and create separate plots
    donefreq=[]

    for freq_group, flux_group, beam_group, date_group in zip(grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates):
        if len(date_group) > 0 and len(flux_group) > 0:  # Only plot if there are valid points
            
            new_valid=[]
 
            for ivalid_sfreqs in valid_sfreqs:
                ivalid_sfreqs=list(ivalid_sfreqs)
                ivalid_sfreqs.append('detect')
                ivalid_sfreqs=tuple(ivalid_sfreqs)
                new_valid.append(ivalid_sfreqs)

            for ivalid_sfreqs in valid_sfreqs2:
                ivalid_sfreqs=list(ivalid_sfreqs)
                ivalid_sfreqs.append('nodetect')
                ivalid_sfreqs=tuple(ivalid_sfreqs)
                new_valid.append(ivalid_sfreqs)



            for valid_data in [new_valid]:          
                #to filter through points for graphing
                filtered_points = [(d, fl, b, f, detect) for f, fl, b, d, detect in valid_data if f in freq_group]
                if filtered_points:
                    filtered_dates, filtered_fluxes, filtered_beams, filtered_freqs, detect = zip(*filtered_points)
                    
                    #filtered_dates, filtered_fluxes, filtered_beams, filtered_freqs = zip(*filtered_points)

                    #for getting into MPc values
                    #filtered_beams = np.array(filtered_beams)
                    #for getting into MPc values
                    #filtered_beams = 10**filtered_beams
                    #filtered_beams = filtered_beams * (np.pi / (180 * 3600))
                    #filtered_beams = filtered_beams * distanceMPc * 1000
                    #filtered_beams = np.log10(filtered_beams)
                    #filtered_beams = filtered_beams.tolist()

                    #filtered_fluxes=10**np.array(filtered_fluxes) * (np.pi / (180 * 3600)) * distanceMPc * 1000
                    #filtered_fluxes=filtered_fluxes.tolist()

            # Assuming filtered_freqs is already defined and is a list or array
            n = len(filtered_freqs)

            freq_diffs = np.full((n, n), np.nan, dtype=object)
            beam_diffs = np.full((n, n), np.nan, dtype=object)
            flux_diffs = np.full((n, n), np.nan, dtype=object)
            date_diffs = np.full((n, n), np.nan, dtype=object)
            detect_diffs = np.full((n, n), np.nan, dtype=object)
            avgfluxij = np.full((n, n), np.nan, dtype=object)


            for i in range(len(filtered_freqs)):
                for j in range(i + 1, len(filtered_freqs)):
                    if i!=j:
                        keep=1
                        if not isinstance(filtered_fluxes[i], list) and not isinstance(filtered_fluxes[j], list):
                            keep=0
                        elif not isinstance(filtered_fluxes[j], list):
                            if filtered_fluxes[j] > filtered_fluxes[i][0]:
                                keep=0
                        elif not isinstance(filtered_fluxes[i], list):
                            if filtered_fluxes[i] > filtered_fluxes[j][0]:
                                keep=0

                        if keep==1:
                            freq_diffs[i, j] = percentage_difference(10**filtered_freqs[i], 10**filtered_freqs[j])
                            if isinstance(filtered_beams[i], list):
                                bothi=filtered_beams[i][1].split(' - ')
                                beami1=float(bothi[0])
                                beami2=float(bothi[1])
                                beamdiff1=percentage_difference(beami1, float(filtered_beams[j]))
                                beamdiff2=percentage_difference(beami2, float(filtered_beams[j]))
                                beam_diffs[i, j] = f"{beamdiff1}-{beamdiff2}"
                            elif isinstance(filtered_beams[j], list):
                                bothj=filtered_beams[j][1].split(' - ')
                                beamj1=float(bothj[0])
                                beamj2=float(bothj[1])
                                beamdiff1=percentage_difference(float(filtered_beams[i]), beamj1)
                                beamdiff2=percentage_difference(float(filtered_beams[i]), beamj2)
                                beam_diffs[i, j] = f"{beamdiff1}-{beamdiff2}"
                            else:
                                beam_diffs[i, j] = percentage_difference(10**filtered_beams[i], 10**filtered_beams[j])


                            if isinstance(filtered_fluxes[i], list) and isinstance(filtered_fluxes[j], list):
                                flux1=10**filtered_fluxes[i][0]
                                flux2=10**filtered_fluxes[j][0]
                                flux1_unc=log_to_linear_uncertainty(filtered_fluxes[i][0],filtered_fluxes[i][1])
                                flux2_unc=log_to_linear_uncertainty(filtered_fluxes[j][0],filtered_fluxes[j][1])
                                flux_diffs[i, j] = [percentage_difference(flux1,flux2),percentage_difference_unc(flux1,flux2,flux1_unc,flux2_unc)]
                                avgfluxij[i, j] = (flux1+flux2)/2 
                            elif isinstance(filtered_fluxes[i], list):
                                flux1=10**filtered_fluxes[i][0]
                                flux2=10**filtered_fluxes[j]
                                flux1_unc=log_to_linear_uncertainty(filtered_fluxes[i][0],filtered_fluxes[i][1])
                                flux2_unc=0
                                flux_diffs[i, j] = [percentage_difference(flux1,flux2)+percentage_difference_unc(flux1,flux2,flux1_unc,flux2_unc)]
                                avgfluxij[i, j] = (flux1+flux2)/2 
                            elif isinstance(filtered_fluxes[j], list):
                                flux1=10**filtered_fluxes[i]
                                flux2=10**filtered_fluxes[j][0]
                                flux1_unc=0
                                flux2_unc=log_to_linear_uncertainty(filtered_fluxes[j][0],filtered_fluxes[j][1])
                                flux_diffs[i, j] = [percentage_difference(flux1,flux2)+percentage_difference_unc(flux1,flux2,flux1_unc,flux2_unc)]
                                avgfluxij[i, j] = (flux1+flux2)/2 



                            if filtered_dates[i]!='NA' and filtered_dates[j]!='NA':
                                date_diffs[i, j] = np.abs(filtered_dates[i]-filtered_dates[j])
                            else:
                                date_diffs[i, j] = np.nan


    allfluxs=[]
    allbeams=[]
    allfreqs=[]
    alldates=[]

    nallfluxs=[]
    nallbeams=[]
    nallfreqs=[]   

    totnum=-1
    for i in range(len(filtered_freqs)):
        for j in range(i + 1, len(filtered_freqs)):
            if not isinstance(beam_diffs[i, j], float):
                continue
            if freq_diffs[i, j] < freq_sim_threshold and beam_diffs[i, j] < beam_sim_threshold:
                    breaker=0
                    newfreq=(10**filtered_freqs[i] + 10**filtered_freqs[j])/2
                    if freqtot[0] <= newfreq <= freqtot[1]:
                        totnum += 1
                        if np.isnan(date_diffs[i, j]):
                            nallfluxs.append(flux_diffs[i, j])
                            nallbeams.append(beam_diffs[i, j])
                            nallfreqs.append(newfreq)
                            input('need to impliment NA dates into variability analysis')
                        else:
                            allfluxs.append(flux_diffs[i, j])
                            allbeams.append(beam_diffs[i, j])
                            alldates.append(date_diffs[i, j])
                            allfreqs.append(newfreq)


    if len(allfreqs)!=0:

        for iallfreqs, iallfluxs, ialldates,iallbeams in zip(allfreqs, allfluxs, alldates, allbeams):
                totfreqslim.append(iallfreqs)
                totdateslim.append(ialldates)
                totdiffslimvar.append(iallfluxs)
                totdiffslimbeam.append(iallbeams)


In [ ]:
def getvaldust(Pf,Pb,alpha=2.5):
    beamrat=(2-Pb)/(2+Pb)
    freqrat=(2-Pf)/(2+Pf)
    #fluxrat=(1-beamrat**2*freqrat**alpha)/((1+beamrat**2*freqrat**alpha)/2)
    fluxrat=(1-freqrat**alpha)/((1+freqrat**alpha)/2)
    return(fluxrat*100)

In [ ]:
#Variability Search: Plot and Print Variability vs Freq
write=True
#write=False
colorbarvar='time'
grapher='variability'
#for variability avg val is flux and for compactness it is beam
#Actually it will give the lower value
#colorbarvar='avgval'
from astroquery.ipac.ned import Ned

source_labels={}

def percentage_difference(x, y):
    return np.abs((x - y) / ((x + y) / 2)) * 100

import ast
from matplotlib.lines import Line2D
from scipy import constants
from astropy import constants as aconstants
import sys
import matplotlib.pyplot as pylab
import os
import shutil
import matplotlib as mpl

allfluxs=[]

def format_val(val):
    if val==0:
        return '0'
    if np.isnan(val):
        return np.nan
    places = max(1,  -int(math.floor(math.log10(abs(val))))+1) 
    #return f'{val:.{places}f}'
    if val >1:
        val=f'{val:.1f}' 
    elif val >0.1:
        val=f'{val:.2f}' 
    elif val >0.01:
        val=f'{val:.3f}'
    elif val >0.001:
        val=f'{val:.4f}'
    else:
        val=f"{val:.{places}f}"
    return val

def format_val2(val):
    if val==0:
        return '0'
    if val<0.1:
        val=f'{val:.4f}'
    elif val<1:
        val=f'{val:.3f}'
    elif val<10:
        val=f'{val:.2f}'
    elif val<100:
        val=f'{val:.1f}'
    else:
        input(f'problem with beam {val}')
    return val

cm2 = pylab.get_cmap('jet')  

def round_down_to_half_integer(num):
    return math.floor(num * 2) / 2

def round_up_to_half_integer(num):
    return math.ceil(num * 2) / 2

pc=aconstants.pc.value
mass_sun=aconstants.M_sun.value
c=constants.c
G=constants.gravitational_constant
pi=constants.pi

threshold=0.0015

txtnamesandcoords='namesandcoords.txt'
path='Beam Mismatch'
if os.path.exists(path):
    shutil.rmtree(path)
os.mkdir(path)

spacings=[13, 8, 8, 10, 9, 9, 10, 10, 10, 15, 15, 15, 10]
tot = 0
for i in spacings:
    tot=tot+i
source='target'

sfreqs1='freq1'
sfreqs2='freq2'
freqper='[sep %]'

sbeams1='beam1'
sbeams2='beam2'
beamper='[sep %]'

sfluxes1='flux1'
sfluxes2='flux2'
fluxper='[sep %]'

sdates1='year1'
sdates2='year2'
datesep='[sep yrs]'

if printall==True:
    corr_table='machinetables/correlations_machine.txt'
elif printall==False:
    if grapher=='variability':
        corr_table='machinetables/variability_machine.txt'
    elif grapher=='compactness':    
        corr_table='machinetables/compactness_machine.txt'

with open(corr_table, 'w') as file:
    if write==True:
        original_stdout = sys.stdout
        sys.stdout = file
    #print(f'{source.ljust(spacings[0])} {sfreqs1.ljust(spacings[1])} {sfreqs2.ljust(spacings[2])} {freqper.ljust(spacings[3])} {sbeams1.ljust(spacings[4])} {sbeams2.ljust(spacings[5])} {beamper.ljust(spacings[6])} {sfluxes1.ljust(spacings[7])} {sfluxes2.ljust(spacings[8])} {fluxper.ljust(spacings[9])} {sdates1.ljust(spacings[10])} {sdates2.ljust(spacings[11])} {datesep.ljust(spacings[12])}')

    newrefs=[]
    indextot=-1
    for isource, data in grouped_data.items():
        if isource!='NGC1194':
            #continue
            pass
        # Main loop to read and process sources from the input file
        if os.path.exists(txtnamesandcoords):
            with open(txtnamesandcoords, 'r') as infile:
                for line in infile:
                    # Parse the line into a list of name-coordinate pairs
                    params = ast.literal_eval(line)
                    for inamecoords in params:              
                        source_name = inamecoords[0][0]
                        if not isource.lower() in source_name.lower():
                            continue
                        source=source_name
                        if source=='CircinusGalaxy':
                            source='Circinus'    
                        ra = inamecoords[1][1]
                        dec = inamecoords[1][2]  # Galaxy Dec in sexagesimal format
                        findvals=process_source(source_name, wantrefs, ra, dec, threshold, inamecoords)
                        distanceMPc=findvals[0][0]
                        distanceMPc_uncer=findvals[0][1]
                        method=findvals[1]
                        ynfindref=findvals[2]
                        refcode=findvals[3]
                        red_depend=findvals[4]
                        if len(findvals[5])!=0:
                            for i in findvals[5]:
                                newrefs.append(i)
        else:
            print(f"File {txtnamesandcoords} does not exist.")

        arcsectokpc= np.pi / (180 * 3600) * distanceMPc * 1000
        arcsectopc=arcsectokpc*1000

        arcsectokpc_error = np.pi / (180 * 3600) * distanceMPc_uncer * 1000
        arcsectopc_unc=arcsectokpc_error*1000

        arcsectopc_fracerror=(arcsectopc_unc/arcsectopc)*100

        distance='NA'
        with open(Greenepath, 'r') as infile:
            j=-1
            breaker=1
            for line in infile:
                j=j+1
                if j>20:
                    continue
                source_name=line.split(' ')[0]
                source_name=source_name.replace("−","-")
                if "WISEA" in isource:
                    isource=isource.split("WISEA")[0]
                if isource.lower() in source_name.lower():
                    log_BH_mass_solar=line.split(' ')[3]
                    log_BH_mass_solar_offset=line.split(' ')[5]
                    breaker=0
            if breaker!=1:
                mass, mass_unc, mass_fracerror, distance, distance_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_unc, diam_bhs_microarcsec_fracerror=getsize(log_BH_mass_solar,log_BH_mass_solar_uncer,distanceMPc,distanceMPc_uncer,arcsectopc,arcsectopc_unc)

        # Filter out elements where date == 'NA'
        valid_sfreqs = [(f, fl, b, d, fn) for f, fl, b, d, fn in zip(data['sfreqs'], data['sfluxs'], data['sbeams'], data['sdates'], data['sfiles'])]
        valid_sfreqs2 = [(f, fl, b, d, fn) for f, fl, b, d, fn in zip(data['sfreqs2'], data['sfluxs2'], data['sbeams2'], data['sdates2'], data['sfiles2'])]
        if len(data['sfreqs']+data['sfreqs2'])==0:
            continue

        # Combine all valid data for grouping
        combined_data = valid_sfreqs + valid_sfreqs2
        
        # Unpack valid data
        if combined_data:
            combined_freqs, combined_fluxes, combined_beams, combined_dates, combined_names = zip(*combined_data)
        else:
            combined_freqs, combined_fluxes, combined_beams, combined_dates, combined_names = [], [], [], [], []

        grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates, grouped_names = [list(combined_freqs)], [list(combined_fluxes)], [list(combined_beams)], [list(combined_dates)], [list(combined_names)]

        #print(grouped_fluxes)
        # Iterate through frequency groups and create separate plots
        donefreq=[]
        for freq_group, flux_group, beam_group, date_group, name_group in zip(grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates, grouped_names):
            if len(date_group) > 0 and len(flux_group) > 0:  # Only plot if there are valid points  

                new_valid=[]
    
                for ivalid_sfreqs in valid_sfreqs:
                    ivalid_sfreqs=list(ivalid_sfreqs)
                    ivalid_sfreqs.append('detect')
                    ivalid_sfreqs=tuple(ivalid_sfreqs)
                    new_valid.append(ivalid_sfreqs)

                for ivalid_sfreqs in valid_sfreqs2:
                    ivalid_sfreqs=list(ivalid_sfreqs)
                    ivalid_sfreqs.append('nodetect')
                    ivalid_sfreqs=tuple(ivalid_sfreqs)
                    new_valid.append(ivalid_sfreqs)


                #for valid_data in new_valid:
                #for valid_data in [
                #    (valid_sfreqs)
                #]:  
                    #filtered_points = [(d, fl, b, f) for f, fl, b, d in valid_data if f in freq_group]

                for valid_data in [new_valid]:
                    filtered_points = [(d, fl, b, f, detect, fn) for f, fl, b, d, detect, fn in valid_data if f in freq_group]
                    if filtered_points:
                        filtered_dates, filtered_fluxes, filtered_beams, filtered_freqs, filenames, detect = zip(*filtered_points)
                        #filtered_dates, filtered_fluxes, filtered_beams, filtered_freqs = zip(*filtered_points)
                        
                        #for getting into MPc values
                        #filtered_beams = np.array(filtered_beams)
                        #filtered_beams = 10**filtered_beams
                        #filtered_beams = filtered_beams * (np.pi / (180 * 3600))
                        #filtered_beams = filtered_beams * distanceMPc * 1000
                        #filtered_beams = np.log10(filtered_beams)
                        #filtered_beams = filtered_beams.tolist()

                        #filtered_fluxes=10**np.array(filtered_fluxes) * (np.pi / (180 * 3600)) * distanceMPc * 1000
                        #filtered_fluxes=filtered_fluxes.tolist()

                # Assuming filtered_freqs is already defined and is a list or array
                n = len(filtered_freqs)

                freq_diffs = np.full((n, n), np.nan, dtype=object)
                avgfreqij = np.full((n, n), np.nan, dtype=object)
                beam_diffs = np.full((n, n), np.nan, dtype=object)
                avgbeamij = np.full((n, n), np.nan, dtype=object)
                flux_diffs = np.full((n, n), np.nan, dtype=object)
                avgfluxij = np.full((n, n), np.nan, dtype=object)
                date_diffs = np.full((n, n), np.nan, dtype=object)
                detect_diffs = np.full((n, n), np.nan, dtype=object)
                name_diffs = np.full((n, n), np.nan, dtype=object)

                totkeep=0

                for i in range(len(filtered_freqs)):
                    for j in range(i + 1, len(filtered_freqs)):
                        keep=1
                        if not isinstance(filtered_fluxes[i], list) and not isinstance(filtered_fluxes[j], list):
                            keep=0
                        elif not isinstance(filtered_fluxes[j], list):
                            if filtered_fluxes[j] > filtered_fluxes[i][0]:
                                keep=0
                        elif not isinstance(filtered_fluxes[i], list):
                            if filtered_fluxes[i] > filtered_fluxes[j][0]:
                                keep=0
                        if i==j:
                            keep=0

                        if keep==0:
                            pass

                        if keep==1:
                            totkeep=totkeep+1
                            freq_diffs[i, j] = percentage_difference(10**filtered_freqs[i], 10**filtered_freqs[j])
                            avgfreqij[i, j] = (10**filtered_freqs[i] + 10**filtered_freqs[j])/2
                            name_diffs[i,j] = [filenames[i],filenames[j]]

                            if isinstance(filtered_beams[i], list):
                                #input('error1')
                                bothi=filtered_beams[i][1].split(' - ')
                                beami1=float(bothi[0])
                                beami2=float(bothi[1])
                                beamdiff1=percentage_difference(beami1, float(filtered_beams[j]))
                                beamdiff2=percentage_difference(beami2, float(filtered_beams[j]))
                                beam_diffs[i, j] = f"{beamdiff1}-{beamdiff2}"
                            elif isinstance(filtered_beams[j], list):
                                #print(filtered_beams[i])
                                #print(filtered_beams[j])
                                #input('error2')
                                bothj=filtered_beams[j][1].split(' - ')
                                beamj1=float(bothj[0])
                                beamj2=float(bothj[1])
                                beamdiff1=percentage_difference(float(filtered_beams[i]), beamj1)
                                beamdiff2=percentage_difference(float(filtered_beams[i]), beamj2)
                                beam_diffs[i, j] = f"{beamdiff1}-{beamdiff2}"
                            else:
                                beam_diffs[i, j] = percentage_difference(10**filtered_beams[i], 10**filtered_beams[j])

                            #avgbeamij[i, j]= (10**filtered_beams[i] + 10**filtered_beams[j])/2
                            if isinstance(filtered_beams[i], list) or isinstance(filtered_beams[j], list):
                                #input('error3')
                                avgbeamij[i, j]='NA'
                            else:
                                if filtered_beams[i] < filtered_beams[j]:
                                    avgbeamij[i, j]= 10**filtered_beams[i]
                                else:
                                    avgbeamij[i, j]= 10**filtered_beams[j]

                            if isinstance(filtered_fluxes[i], list) and isinstance(filtered_fluxes[j], list):
                                flux1=10**filtered_fluxes[i][0]
                                flux2=10**filtered_fluxes[j][0]
                                flux1_unc=log_to_linear_uncertainty(filtered_fluxes[i][0],filtered_fluxes[i][1])
                                flux2_unc=log_to_linear_uncertainty(filtered_fluxes[j][0],filtered_fluxes[j][1])
                                flux_diffs[i, j] = [percentage_difference(flux1,flux2),percentage_difference_unc(flux1,flux2,flux1_unc,flux2_unc)]
                                avgfluxij[i, j] = (flux1+flux2)/2 
                            elif isinstance(filtered_fluxes[i], list):
                                flux1=10**filtered_fluxes[i][0]
                                flux2=10**filtered_fluxes[j]
                                flux1_unc=log_to_linear_uncertainty(filtered_fluxes[i][0],filtered_fluxes[i][1])
                                flux2_unc=0
                                flux_diffs[i, j] = [percentage_difference(flux1,flux2)+percentage_difference_unc(flux1,flux2,flux1_unc,flux2_unc)]
                                avgfluxij[i, j] = (flux1+flux2)/2 
                            elif isinstance(filtered_fluxes[j], list):
                                flux1=10**filtered_fluxes[i]
                                flux2=10**filtered_fluxes[j][0]
                                flux1_unc=0
                                flux2_unc=log_to_linear_uncertainty(filtered_fluxes[j][0],filtered_fluxes[j][1])
                                flux2=flux2+flux2_unc
                                flux_diffs[i, j] = [percentage_difference(flux1,flux2)]
                                avgfluxij[i, j] = (flux1+flux2)/2 

                            if filtered_dates[i]!='NA' and filtered_dates[j]!='NA':
                                date_diffs[i, j] = np.abs(filtered_dates[i]-filtered_dates[j])
                            else:
                                date_diffs[i, j] = 'NA'

        alldiffs=[]
        allfluxavg=[]

        alldiffunchi=[]
        alldiffunchi2=[]
        alldiffunclo=[]

        allbeams=[]
        allbeamavg=[]

        allfreqs=[]
        alldates=[]

        allnames=[]

        nalldiffs=[]
        nallfluxavg=[]

        nallbeams=[]
        nallbeamavg=[]

        nallfreqs=[]   

        nallnames=[]

        totnum=0
        if filtered_freqs=='NA':
            continue
        for i in range(len(filtered_freqs)):
            for j in range(i + 1, len(filtered_freqs)):
                if printall==False:
                    
                    breaker=0
                    #if fluxes cannot be known than you cant compare fluxes for y axis variability or ensure they are the same for compactness
                    if flux_diffs[i, j]=='NA':
                        breaker=1
                    #if beams are not known that you cant compare beam sizes for y axis compactness or ensure they are the same for variability
                    if beam_diffs[i, j]=='NA' or isinstance(beam_diffs[i,j],str):
                        breaker=1
                        
                    if breaker==1:
                        continue

                    if freqthresh==True:
                        if not freq_diffs[i, j] < freq_sim_threshold:
                            breaker=1
                    if beamthresh==True:
                        if not beam_diffs[i, j] < beam_sim_threshold:
                            breaker=1
 
                    if breaker==1:
                        continue

                    indextot+=1  
                    if printall==False and indextot!=0 and totnum!=0:
                        print(f'{source.ljust(spacings[0])} {sfreqs1.ljust(spacings[1])} {sfreqs2.ljust(spacings[2])} {freqper.ljust(spacings[3])} {sbeams1.ljust(spacings[4])} {sbeams2.ljust(spacings[5])} {beamper.ljust(spacings[6])} {sfluxes1.ljust(spacings[7])} {sfluxes2.ljust(spacings[8])} {fluxper.ljust(spacings[9])} {sdates1.ljust(spacings[10])} {sdates2.ljust(spacings[11])} {datesep.ljust(spacings[12])}')
                        if '±' in fluxper:
                            both=((fluxper.replace('[','').replace(']',''))).split('±')
                            flux=both[0]
                            totunc=float(flux)/(fluxunc+float(both[1]))
                        pass
                newfreq=(10**filtered_freqs[i] + 10**filtered_freqs[j])/2
                if freqtot[0] <= newfreq <= freqtot[1]:
                    totnum += 1
                    if date_diffs[i, j]=='NA' and colorbarvar=='time':
                        nalldiffs.append(flux_diffs[i, j])
                        nallfluxavg.append(avgfluxij[i, j])

                        nallbeams.append(beam_diffs[i, j])
                        nallbeamavg.append(avgbeamij[i, j])

                        nallnames.append(name_diffs[i,j])

                        nallfreqs.append(newfreq)

                        sfreqs1=10**filtered_freqs[i]
                        if sfreqs1<10:
                            sfreqs1=f'{sfreqs1:.1f}'
                        else:
                            sfreqs1=f'{sfreqs1:.0f}'

                        sfreqs2=10**filtered_freqs[j]
                        if sfreqs2<10:
                            sfreqs2=f'{sfreqs2:.1f}'
                        else:
                            sfreqs2=f'{sfreqs2:.0f}'

                        if freq_diffs[i, j]<1:
                            freqper='[<1]'
                        else:
                            freqper='['+format_val(freq_diffs[i, j])+']'

                        beam1=10**filtered_beams[i]
                        beam2=10**filtered_beams[j]
                        sbeams1=format_val2(beam1)
                        sbeams2=format_val2(beam2)

                        if beam_diffs[i,j]<1:
                            beamper='[<1]'
                        else:
                            beamper='['+format_val(beam_diffs[i, j])+']'

                        if detect[i]=='nodetect':
                            fluxes1=f'{10**filtered_fluxes[i]:.1f}*'
                        else:
                            fluxes1=f'{10**filtered_fluxes[i]:.1f}'

                        if detect[j]=='nodetect':
                            fluxes2=f'{10**filtered_fluxes[j]:.1f}*'
                        else:
                            fluxes2=f'{10**filtered_fluxes[j]:.1f}'
                        
                        if flux_diffs[i, j] == 'NA':
                            fluxper = '[NA]'
                        else:
                            if flux_diffs[i, j][0]<1:
                                fluxper='[<1]'
                            else:
                                if len(flux_diffs[i, j])==2:
                                    fluxper='['+format_val(flux_diffs[i, j][0]) + '±' + format_val(flux_diffs[i, j][1]) + ']'
                                else:
                                    fluxper='[>'+format_val(flux_diffs[i, j][0])+']'

                        if flux_diffs[i, j]!=0:
                            allfluxs.append(flux_diffs[i, j])

                        if filtered_dates[i] =='NA' and filtered_dates[j] =='NA':
                            sdates1='[NA]'
                            sdates2='[NA]'
                        elif filtered_dates[i] =='NA':
                            sdates1='[NA]'
                            sdates2=f'{decimal_year_to_date(filtered_dates[j])}'
                        elif filtered_dates[j] =='NA':
                            sdates1=f'{decimal_year_to_date(filtered_dates[i])}'
                            sdates2='[NA]'
                        datesep='[NA]'
                        #print(f'{source.ljust(spacings[0])} {sfreqs1.ljust(spacings[1])} {sfreqs2.ljust(spacings[2])} {freqper.ljust(spacings[3])} {sbeams1.ljust(spacings[4])} {sbeams2.ljust(spacings[5])} {beamper.ljust(spacings[6])} {sfluxes1.ljust(spacings[7])} {sfluxes2.ljust(spacings[8])} {fluxper.ljust(spacings[9])} {sdates1.ljust(spacings[10])} {sdates2.ljust(spacings[11])} {datesep.ljust(spacings[12])}')
                    else:
                        alldiffs.append(flux_diffs[i, j])
                        allfluxavg.append(avgfluxij[i, j])

                        allbeams.append(beam_diffs[i, j])
                        allbeamavg.append(avgbeamij[i, j])

                        alldates.append(date_diffs[i, j])
                        allfreqs.append(newfreq)

                        allnames.append(name_diffs[i,j])
                        #print(allnames[0][0])
                        #print(allnames[0][1])



                        #flux1 is fluxi and flux2 is fluxj
                        '''
                        if flux1>flux2:
                            lowfreq=10**filtered_freqs[j]
                            lowflux=flux2
                            highfreq=10**filtered_freqs[i]  
                            highflux=flux1
                            lowbeam=10**filtered_beams[j]
                            highbeam=10**filtered_beams[i]
                        elif flux1<flux2:
                            lowfreq=10**filtered_freqs[i]
                            lowflux=flux1
                            highfreq=10**filtered_freqs[j]   
                            highflux=flux2
                            lowbeam=10**filtered_beams[i]
                            highbeam=10**filtered_beams[j]  
                        else:
                            input(f'{filtered_freqs[i]} {filtered_freqs[j]}')
                        '''

                        flux1=10**filtered_fluxes[i][0]
                        flux2=10**filtered_fluxes[j][0]

                        freq1=10**filtered_freqs[i]
                        freq2=10**filtered_freqs[j]


                        freq1todust=flux1*(freq2/freq1)**(2.5)
                        freq2todust=flux2*(freq1/freq2)**(2.5)

                        diff1dust=percentage_difference(freq1todust, flux2)
                        diff2dust=percentage_difference(freq2todust, flux1)

                        freq1tosynch=flux1*(freq2/freq1)**(-0.5)  
                        freq2tosynch=flux2*(freq1/freq2)**(-0.5)  
                        diff1synch=percentage_difference(freq1tosynch, flux2)
                        diff2synch=percentage_difference(freq2tosynch, flux1)

                        dustdiff=f'{diff1dust:.1f} - {diff2dust:.1f}'
                        synchdiff=f'{diff1synch:.1f} - {diff2synch:.1f}'


                        sfreqs1=freq1
                        if sfreqs1<10:
                            sfreqs1=f'{sfreqs1:.1f}'
                        else:
                            sfreqs1=f'{sfreqs1:.0f}'

                        sfreqs2=freq1
                        if sfreqs2<10:
                            sfreqs2=f'{sfreqs2:.1f}'
                        else:
                            sfreqs2=f'{sfreqs2:.0f}'

                        freqfrac_sys=freq_diffs[i, j]/100
                        beamfrac_sys=beam_diffs[i, j]/100

                        fluxunc=getvaldust(freqfrac_sys,beamfrac_sys)

                        if len(flux_diffs[i, j])==2:
                            lowbound = False
                            fluxunc_inst=flux_diffs[i, j][1]
                            fluxunc1 = fluxunc_inst + fluxunc
                            fluxunc2 = fluxunc_inst - fluxunc

                            fluxunclo=linear_to_log_uncertainty(flux_diffs[i, j][0],fluxunc_inst)
                            fluxunchi=linear_to_log_uncertainty(flux_diffs[i, j][0],fluxunc1)
                            fluxunchi2=linear_to_log_uncertainty(flux_diffs[i, j][0],fluxunc2)

                            alldiffunclo.append(fluxunclo)
                            alldiffunchi.append(fluxunchi)
                            alldiffunchi2.append(fluxunchi2)
                        else:
                            lowbound=True
                            fluxunc_inst='xNA'
                            fluxunc1 = fluxunc_inst + fluxunc
                            fluxunc2 = fluxunc_inst - fluxunc

                            fluxunclo='xNA'
                            fluxunchi=linear_to_log_uncertainty(flux_diffs[i, j][0],fluxunc1)
                            fluxunchi2=linear_to_log_uncertainty(flux_diffs[i, j][0],fluxunc2)

                            alldiffunclo.append(fluxunclo)
                            alldiffunchi.append(fluxunchi)
                            alldiffunchi2.append(fluxunchi2)
                            
                        if freq_diffs[i, j]<1:
                            freqper='[<1]'
                        else:
                            freqper='['+format_val(freq_diffs[i, j])+']'

                        if isinstance(filtered_beams[i], list):
                            bothi=filtered_beams[i][1].split(' - ')
                            beami1=10**float(bothi[0])
                            beami2=10**float(bothi[1])
                            sbeams1=f"{format_val2(beami1)}-{format_val2(beami2)}"
                        else:
                            beam1=10**filtered_beams[i]
                            sbeams1=format_val2(beam1)


                        if isinstance(filtered_beams[j], list):
                            bothj=filtered_beams[j][1].split(' - ')
                            beamj1=10**float(bothj[0])
                            beamj2=10**float(bothj[1])
                            sbeams2=f"{format_val2(beamj1)}-{format_val2(beamj2)}"
                        else:
                            beam2=10**filtered_beams[j]
                            sbeams2=format_val2(beam2)

                        if isinstance(beam_diffs[i, j], float):
                            if beam_diffs[i, j]<1:
                                beamper='[<1]'
                            else:
                                beamper='['+format_val(beam_diffs[i, j])+']'
                        else:
                            beamper='[NA]'

                        if detect[i]=='nodetect':
                            fluxes1=f'{10**filtered_fluxes[i]:.1f}*'
                        else:
                            fluxes1=f'{10**filtered_fluxes[i][0]:.1f}'

                        if detect[j]=='nodetect':
                            fluxes2=f'{10**filtered_fluxes[j]:.1f}*'
                        else:
                            fluxes2=f'{10**filtered_fluxes[j][0]:.1f}'
                            
                        sfluxes1=f'{fluxes1}'
                        sfluxes2=f'{fluxes2}'

                        if flux_diffs[i, j]!=0:
                            allfluxs.append(flux_diffs[i, j])
                        if flux_diffs[i, j]!=0 and i+1!=j:
                            #print(i)
                            #print(j)
                            #input('error ij')
                            pass

                        #print(filtered_dates[i])
                        #print(filtered_dates[j])
                        sdates1=f'{decimal_year_to_date(filtered_dates[i])}'
                        sdates2=f'{decimal_year_to_date(filtered_dates[j])}'
                        if date_diffs[i, j]!='NA':
                            idate_diffs=f"{date_diffs[i, j]:.3f}"
                        else:
                            idate_diffs='NA'
                        datesep=f'['+idate_diffs+']'
                        if flux_diffs[i, j] == 'NA':
                            fluxper = '[NA]'
                        else:
                            if flux_diffs[i, j][0]<1:
                                fluxper='[<1]'
                            else:
                                if len(flux_diffs[i, j])==2:
                                    fluxper='['+format_val(flux_diffs[i, j][0]) + '±' + format_val(flux_diffs[i, j][1]) + ']'
                                else:
                                    fluxper='[>'+format_val(flux_diffs[i, j][0])+']'

                        #print(dustdiff)
                        #print(synchdiff)
                        #print(fluxper)
                        '''
                        print(sfreqs1)
                        print(sfreqs2)
                        print(freqper)
                        print(sbeams1)
                        print(sbeams2)
                        print(beamper)
                        print(sfluxes1)
                        print(sfluxes2)
                        print(fluxper)
                        print(sdates1)
                        print(sdates2)  
                        print(datesep)
                        '''
            
                    if printall==True:
                        indextot+=1 
                        print(f'{isource.ljust(spacings[0])} {sfreqs1.ljust(spacings[1])} {sfreqs2.ljust(spacings[2])} {freqper.ljust(spacings[3])} {sbeams1.ljust(spacings[4])} {sbeams2.ljust(spacings[5])} {beamper.ljust(spacings[6])} {sfluxes1.ljust(spacings[7])} {sfluxes2.ljust(spacings[8])} {fluxper.ljust(spacings[9])} {sdates1.ljust(spacings[10])} {sdates2.ljust(spacings[11])} {datesep.ljust(spacings[12])}')

        if len(allfreqs)!=0:
            #gets the most recent values:

                        
            if printall==False:
                print(f'{source.ljust(spacings[0])} {sfreqs1.ljust(spacings[1])} {sfreqs2.ljust(spacings[2])} {freqper.ljust(spacings[3])} {sbeams1.ljust(spacings[4])} {sbeams2.ljust(spacings[5])} {beamper.ljust(spacings[6])} {sfluxes1.ljust(spacings[7])} {sfluxes2.ljust(spacings[8])} {fluxper.ljust(spacings[9])} {sdates1.ljust(spacings[10])} {sdates2.ljust(spacings[11])} {datesep.ljust(spacings[12])}')
            cm2 = pylab.get_cmap('jet')  


            totdates = np.array(alldates)
            totfreqs = np.array(allfreqs)

            #note these are note averages they are the lower of the 2 values
            allfluxavg = np.array(allfluxavg)
            allbeamavg = np.array(allbeamavg)

            # Create a boolean mask where all fields are valid (i.e., not 'NA')
            valid_mask = [
                (d != 'NA' and f != 'NA' and fl != 'NA' and b != 'NA')
                for d, f, fl, b in zip(totdates, totfreqs, allfluxavg, allbeamavg)
            ]

            if np.size(totdates[valid_mask]) == 0:
                continue

            # Apply the mask to each array
            totdates = totdates[valid_mask].astype(float)
            totfreqs = totfreqs[valid_mask].astype(float)
            allfluxavg = allfluxavg[valid_mask].astype(float)
            allbeamavg = allbeamavg[valid_mask].astype(float)

            allfluxavg=np.log10(allfluxavg.astype(float))
            allbeamavg=np.log10(allbeamavg.astype(float))

            minfreq=np.log10(np.min(totfreqslim))
            maxfreq=np.log10(np.max(totfreqslim))

            interval = maxfreq - minfreq
            ten_percent = interval * 0.05
            #minfreq = minfreq - ten_percent
            #maxfreq = maxfreq + ten_percent

            #mindate=np.min(totdateslim)
            #maxdate=np.max(totdateslim)
            #mindate=np.min(totdates)
            mindate=0
            maxdate=np.max(totdates)
            if maxdate<5:
                maxdate=5

            #interval = maxdate - mindate
            #ten_percent = interval * 0.05
            #mindate = mindate - ten_percent
            #maxdate = maxdate + ten_percent

            minfluxavg=np.min(allfluxavg)
            #if minfluxavg>0:
            #    minfluxavg=0
            maxfluxavg=np.max(allfluxavg)
            #if maxfluxavg<1:
            #    maxfluxavg=1

            interval = maxfluxavg - minfluxavg
            ten_percent = interval * 0.05
            minfluxavg = minfluxavg - ten_percent
            maxfluxavg = maxfluxavg + ten_percent

            minbeamavg=np.min(allbeamavg)
            #if minbeamavg>0:
            #    minbeamavg=0
            maxbeamavg=np.max(allbeamavg)

            interval = maxbeamavg - minbeamavg
            ten_percent = interval * 0.05
            minbeamavg = minbeamavg - ten_percent
            maxbeamavg = maxbeamavg + ten_percent

            minfreqp=round_down_to_half_integer(minfreq)
            maxfreqp=round_up_to_half_integer(maxfreq)

            if not math.isnan(mindate):
                mindatep=round_down_to_half_integer(mindate)
            if not math.isnan(maxdate):
                maxdatep=round_up_to_half_integer(maxdate)

            if not math.isnan(minfluxavg):
                minfluxavgp=round_down_to_half_integer(minfluxavg)
            if not math.isnan(maxfluxavg):
                maxfluxavgp=round_up_to_half_integer(maxfluxavg)

            if not math.isnan(minbeamavg):
                minbeamavgp=round_down_to_half_integer(minbeamavg)
            if not math.isnan(maxbeamavg):
                maxbeamavgp=round_up_to_half_integer(maxbeamavg)

            if grapher=='variability':
                #all diffs are flux differences and are on the y axis
                totdiffs=alldiffs

            elif grapher=='compactness':
                #all diffs are beam differences and are on the y axis
                totdiffs=allbeams

            if colorbarvar=='time':
                minnorm=mindatep
                #minnorm=0
                maxnorm=maxdatep
                if maxnorm<5:
                    maxnorm=5
                totnorm=totdates
            elif colorbarvar=='avgval':
                if grapher=='variability':
                    minnorm=minfluxavgp
                    maxnorm=maxfluxavgp
                    totnorm=allfluxavg
                elif grapher=='compactness':
                    minnorm=minbeamavgp
                    maxnorm=maxbeamavgp
                    totnorm=allbeamavg

            norm=matplotlib.colors.Normalize(vmin=minnorm, vmax=maxnorm)
            scalar_map = cmx.ScalarMappable(norm=norm, cmap=cm2)
            colors = scalar_map.to_rgba(totnorm)
            fig, ax = pylab.subplots(dpi=300)
            #pylab.scatter(np.log10(totfreqs), np.log10(totdiffs), marker='o', s=5, c=colors)  # You can change the color and marker style
            
            log_freqs = np.log10(totfreqs)
            totnorm[totnorm == 0] = 1/365
            totnorm = np.log10(totnorm)

            precthresh=1
            precthresh=np.log10(precthresh)

            # Place a lower arrow for points that have the same flux(for variability) or beam (for compactness)
            
            valid_mask = []
            for x, y, z, k1, k2, k3 in zip(totdiffs, log_freqs, colors, alldiffunchi, alldiffunclo, alldiffunchi2):
                def is_invalid(val):
                        return (isinstance(val, str) and ('-' in val or val == 'NA')) or np.ndim(val) > 1

                if is_invalid(x) or is_invalid(y) or is_invalid(z):
                        valid_mask.append(False)
                else:
                        valid_mask.append(True)

            # Apply the validated mask
            totdiffs = [float(f[0]) for f in totdiffs]
            filtered_diffs = np.array([float(x) for x, keep in zip(totdiffs, valid_mask) if keep])
            filtered_freqs = np.array([y for y, keep in zip(log_freqs, valid_mask) if keep])
            filtered_colors = np.array([z for z, keep in zip(colors, valid_mask) if keep])
            filtered_diffsunchi = np.array([k for k, keep in zip(alldiffunchi, valid_mask) if keep])
            filtered_diffsunchi2 = np.array([k for k, keep in zip(alldiffunchi2, valid_mask) if keep])
            filtered_diffsunclo = np.array([k for k, keep in zip(alldiffunclo, valid_mask) if keep])
            filtered_times = np.array([y for y, keep in zip(totnorm, valid_mask) if keep])


            log_diffs = np.maximum(np.log10(np.maximum(filtered_diffs, 1e-10)), precthresh)

            normal_mask = log_diffs > precthresh

            # Now your original logic applies correctly:
            normal_x = filtered_freqs[normal_mask]
            normal_y = log_diffs[normal_mask]
            #normal_color = filtered_colors[normal_mask]
            normal_color = normal_x
            normal_yunchi = filtered_diffsunchi[normal_mask]
            normal_yunchi2 = filtered_diffsunchi[normal_mask]
            normal_yunclo = filtered_diffsunclo[normal_mask]
            normal_x_times = filtered_times[normal_mask]

            adjusted_x = filtered_freqs[~normal_mask]
            adjusted_y = log_diffs[~normal_mask]
            #adjusted_color = filtered_colors[~normal_mask]
            adjusted_color = adjusted_x
            adjusted_x_times = []
            adjusted_x_times = filtered_times[~normal_mask]
            

            if len(nallfreqs)!=0:
                ntotfreqs=nallfreqs
                ntotdiffs=nalldiffs


            #normal_x = log_freqs
            #normal_y = log_diffs
            #normal_color = colors
            #print(f'x: {normal_x}')
            #print(f'y: {normal_y}')
            #print(f'c: {totnorm}')

            vmin = 0
            vmax = np.log10(500)

            # round down to nearest tenth
            vmin_rounded = math.floor(vmin * 10) / 10.0

            # round up to nearest tenth
            vmax_rounded = math.ceil(vmax * 10) / 10.0

            cmap = mpl.cm.viridis
            norm = mpl.colors.Normalize(vmin=vmin_rounded,
                                        vmax=vmax_rounded)
            sm = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)  # for the colorbar

            for x, y, ylo, yhi, yhi2, c in zip(normal_x_times, normal_y, normal_yunclo ,normal_yunchi,normal_yunchi2 , normal_color):
                #ylo is the error due to flux uncertainty, and yhi is the error due to incoporating frequency effects
                #ylo is positive... but yhi may be lower than ylo, and could in fact even be negative
                if not y/ylo>3:
                    continue
                
                col = cmap(norm(c))
                if ylo=='xNA':
                    if c>2.3:
                      input('error')
                    else:  
                        shape = '^'

                    if yhi-y<=0:
                          pylab.scatter(x, y,
                                      facecolors='none', edgecolors=col,
                                      marker=shape, s=40)
                    else:
                          if yhi2>yhi:
                              yhival=yhi2
                          else:
                              yhival=yhi
                          ax.errorbar(x, y,
                                      yerr=[[yhival], [0]],    
                                      fmt=shape, ms=5, linestyle='none',
                                      markerfacecolor='none', markeredgecolor=col, ecolor=col, capsize=0)
                          
                          cap_len = 1 
                          ax.hlines(y + yhi, x - cap_len/2, x + cap_len/2, colors=col, linewidth=0.5)
                    continue
                

                if c>2.3:
                    shape='o'
                else:
                    shape='s'
                
                if y-ylo > 0:
                    if yhi>ylo:
                        yplot=yhi
                    else:
                        yplot=ylo
                    ax.errorbar(x, y,
                                yerr=[[yplot], [yplot]],     # asymmetric: lower=ylo, upper=0
                                fmt=shape, ms=5, linestyle='none',
                                markerfacecolor='none', markeredgecolor=col, ecolor=col, capsize=0)
                    cap_len = 1   # length in data units (tweak as needed)
                    ax.hlines(y - yhi2, x - cap_len/2, x + cap_len/2, colors=col, linewidth=0.5,linestyles='--')
                    ax.hlines(y + yhi, x - cap_len/2, x + cap_len/2, colors=col, linewidth=0.5,linestyles='--')

                    # Now add custom horizontal caps
                    cap_len = 1  # length in data units (adjust to taste)
                    ax.hlines(y - ylo, x - cap_len/2, x + cap_len/2, colors=col, linewidth=0.5)  # lower cap
                    ax.hlines(y + ylo, x - cap_len/2, x + cap_len/2, colors=col, linewidth=0.5)  # lower cap
                else:
                    input('error ylo')
                    #ax.errorbar(x, y,
                    #yerr=[[y], [ylo]],     # asymmetric: lower=ylo, upper=0
                    #fmt=shape, ms=5, linestyle='none',
                    #markerfacecolor='none', markeredgecolor=col, ecolor=col, capsize=4, elinewidth=0.5)
                    #ecolor=col

                    # add downward arrow at bottom of error bar
                    #ax.scatter(x, 0, marker='v', color=col, s=40)   # 'v' = downward triangle
                    #ax.scatter(x, yhi-y, marker='_', color=col, s=120)   # 'v' = downward triangle
                    #ax.scatter(x, y+yhi, marker='_', color=col, s=120)   # 'v' = downward triangle
            

            normal_yunchi_clean = [0 if v == 'NA' else v for v in normal_yunchi]
            normal_yunchi_clean2 = [0 if v == 'NA' else v for v in normal_yunchi2]
            normal_yunclo_clean = [0 if v == 'NA' else v for v in normal_yunclo]

            yhi_arr = np.asarray(normal_yunchi_clean, dtype=float)
            yhi2_arr = np.asarray(normal_yunchi_clean, dtype=float)
            ylo_arr = np.asarray(normal_yunclo_clean, dtype=float)


            y_arr   = np.asarray(normal_y,       dtype=float)
            c_arr   = np.asarray(normal_color,   dtype=float)

            # Base mask (SNR > 3 and finite)
            base_mask = np.isfinite(y_arr) & np.isfinite(yhi_arr) & np.isfinite(yhi2_arr) & np.isfinite(ylo_arr) & ((y_arr / ylo_arr) > 3 )


            groups = [
                (c_arr < 2.3,  "c < 2.3",  'black'),
                (c_arr >= 2.3, "c ≥ 2.3",  'red'),
            ]

            y_top_hi_black = y_top_lo_black = np.nan
            y_top_hi_red   = y_top_lo_red   = np.nan

            for cond, label, col in groups:
                mask = base_mask & cond
                if np.any(mask):
                    #yhi is with includes frequenc effects
                    #ylow does not

                    y_top_hi_array=[]
                    y_top_lo_array=[]

                    y_top_hi2_array=[]


                    for iy, iyhi, iyhi2, iylo in zip(y_arr[mask], yhi_arr[mask], yhi2_arr[mask],  ylo_arr[mask]):
                        iyhi_lin = log_to_linear_uncertainty(iy, iyhi)
                        iyhi2_lin = log_to_linear_uncertainty(iy, iyhi2)
                        iylo_lin = log_to_linear_uncertainty(iy, iylo)
                        iy_lin = 10**iy

                        y_top_hi_array.append([iy_lin+iyhi_lin,iy+iyhi])

                        y_top_hi2_array.append([iy_lin+iyhi2_lin,iy+iyhi2])

                        y_top_lo_array.append([iy_lin+iylo_lin,iy+iylo])


                    
                    y_top_hi, logytop_hi = np.nanmax(np.array(y_top_hi_array), axis=0)
                    y_top_lo, logytop_lo = np.nanmax(np.array(y_top_lo_array), axis=0)

                      
                    if col == 'black':   # <200 GHz
                        y_top_hi_black, y_top_lo_black = y_top_hi, y_top_lo
                    elif col == 'red':   # ≥200 GHz
                        y_top_hi_red, y_top_lo_red = y_top_hi, y_top_lo


                    # hi → solid line
                    ax.axhline(logytop_hi, color=col, linestyle='--', linewidth=0.6, zorder=10,
                            label=f"max hi {label}")

                    # lo → dashed line
                    ax.axhline(logytop_lo, color=col, linestyle='-', linewidth=0.6, zorder=10,
                            label=f"max lo {label}")


            cbar = pylab.colorbar(sm, ax=ax)


            #pylab.scatter(normal_x, normal_y, c=normal_color, marker='o', s=5, )
            pylab.grid(True)
                

            for x, y, c in zip(adjusted_x_times, adjusted_y, adjusted_color):
                col = cmap(norm(c))
                if c>2.3:
                    input('error')
                    #shape = 'D' if c > 2.3 
                else:
                    shape='v'
                pylab.scatter(x, y,
                            facecolors='none', edgecolors=col,
                            marker=shape, s=40)


            pylab.xlabel(r'log$_{10}$ Time Scale [years]')


            legend_elements = []
            
            # figure out which markers were used
            has_diamond = any(c <= 2.3 for c in adjusted_color)
            has_triangle = any(c > 2.3 for c in adjusted_color)

            if np.isfinite(y_top_hi_black):
                legend_elements.append(Line2D([0], [0], marker='s', color='black', markerfacecolor='none',
                    markersize=6, linestyle='None', label='<200GHz'))
                
            if has_diamond:
                legend_elements.append(
                    Line2D([0], [0], marker='v', linestyle='None',
                        markerfacecolor='none', markeredgecolor='black',
                        markersize=6, label='<200GHz No Variability (<1%)')
                )
            if np.isfinite(y_top_lo_black):
                legend_elements.append(
                    Line2D([0], [0], color='black', linewidth=1, linestyle='-',
                        label=fr'<200GHz Top Bound Variability $\alpha=0$: {y_top_lo_black:.1f}%')
                )
            if np.isfinite(y_top_hi_black):
                legend_elements.append(
                    Line2D([0], [0], color='black', linewidth=1, linestyle='--',
                        label=fr'<200GHz Top Bound Variability $\alpha=+2.5$: {y_top_hi_black:.1f}%')
                )
            if np.isfinite(y_top_hi_red):
                legend_elements.append(Line2D([0], [0], marker='o', color='red', markerfacecolor='none',
                    markersize=6, linestyle='None', label='>200GHz'))
            if has_triangle:
                legend_elements.append(
                    Line2D([0], [0], marker='D', linestyle='None',
                        markerfacecolor='none', markeredgecolor='black',
                        markersize=6, label='>200GHz No Variability (<1%)')
                )
            if np.isfinite(y_top_lo_red):
                legend_elements.append(
                    Line2D([0], [0], color='red', linewidth=1, linestyle='-',
                        label=fr'>200GHz Top Bound Variability $\alpha=0$: {y_top_lo_red:.1f}%')
                )
            if np.isfinite(y_top_hi_red):
                legend_elements.append(
                    Line2D([0], [0], color='red', linewidth=1, linestyle='--',
                        label=fr'>200GHz Top Bound Variability $\alpha=+2.5$: {y_top_hi_red:.1f}%')
                )

            ax.legend(handles=legend_elements, loc="best", frameon=False, fontsize=7)
            
            # inside your loop, after legend_elements is built
            if source not in source_labels:
                source_labels[source] = []

            for h in legend_elements:
                label = h.get_label()
                if ':' in label:  # only keep the ones with values
                    source_labels[source].append(label)


        
            if grapher=='variability':
                pylab.title(f'{source} Variability')
                pylab.ylabel(r'$\log_{10}\ \mathrm{Flux\ [mJy]\ [Beam]^{-1}}\ \mathrm{Difference\ [\%]}$')
            elif grapher=='compactness':
                pylab.title(f'{source} Compactness')
                pylab.ylabel(r'log$_{10}$ Beam Size ["] Difference [%]')

            x_tick_positions = np.arange(-3,1.6, 0.5)  
            #x_tick_positions = np.arange(minfreqp, maxfreqp + 0.1 , 0.1) 
            x_tick_labels =  [f"{pos:.2f}" for pos in x_tick_positions] 

            cbar.set_label(r'log$_{10}$ Frequency [GHz]')  # <-- put your label here

            # Suppose you want a tick every 5 units
            major_ticks = np.arange(vmin_rounded, vmax_rounded, 1)
            cbar.set_ticks(major_ticks)

            # (Optional) format the labels
            cbar.set_ticklabels([str(t) for t in major_ticks])


            # generate candidate minor ticks
            step = (major_ticks[1] - major_ticks[0]) / 5
            minor_tick_locations_date = np.arange(vmin_rounded, vmax_rounded, step)

            # get the major ticks from the colorbar
            major_ticks = cbar.get_ticks()

            # filter out anything already a major tick
            filtered_minor_ticks = [tick for tick in minor_tick_locations_date 
                                    if tick not in major_ticks]

            # apply as minor ticks
            cbar.ax.yaxis.set_ticks(filtered_minor_ticks, minor=True)


            #ax.set_xlim([-2.7, 1])
            ax.set_xticks(x_tick_positions)

            y_tick_positions = np.arange(0, 2.1 , 0.5) 
            #y_tick_positions = np.arange(miny, maxy + 0.1 , 0.1) 
            y_tick_labels = [f"{pos:.2f}" for pos in y_tick_positions]  

            #ax.set_ylim([-0.1, 1.8]) 
            ax.set_yticks(y_tick_positions)
            ax.set_yticklabels(y_tick_labels)

            minor_locatorx = MultipleLocator(0.1)
            ax.xaxis.set_minor_locator(minor_locatorx)
            minor_locatory = MultipleLocator(0.1)
            ax.yaxis.set_minor_locator(minor_locatory)

            graphtype='Variability'

            nicename=f"{source}_{graphtype}.pdf"
            if not os.path.exists(f'{graphtype}Graphs'):
                os.makedirs(f'{graphtype}Graphs')
            if os.path.exists(f'{graphtype}Graphs/{nicename}'):
                os.remove(f'{graphtype}Graphs/{nicename}')
            if printall==False:
                pylab.savefig(f'{graphtype}Graphs/{nicename}', bbox_inches='tight', dpi=300)
                pylab.savefig(f'{graphtype}Graphs/{nicename}', dpi=300)
                pylab.show()
            else:
             pylab.clf() 

    if write==True:
        sys.stdout = original_stdout


In [ ]:
for source, labels in source_labels.items():
    print(source, labels)


In [ ]:
#circinus
val=14.54/(14.54+0.00132)*13.3+0.00132/(14.54+0.00132)*27.9
val=val/100
val=(2+val)/(2-val)
print(val)
print(val*14.54-14.54)
print(14.54-14.54/val)
